# 🚆Train_station
지하철 정류장 데이터 전처리

### TODO
- [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
- [X] 결측값 확인
  - [X] 위경도 결측값 분석
  - [X] 결측값 채워진 데이터셋 구하기
  - [X] 노선명 통일
  - [X] 역사명 통일
- [X] 두 데이터셋 하나로 결합
  - [X] 역사명, 노선명 통일 
  - [X] 노선명을 지하철 호선명으로 통일, 중복역 확인
  - [X] 위경도 이상값 확인 
  - [X] 위경도 결측값 재확인
- [X] 지도에 시각화
  - [X] 이상값 확인
- [X] 역번호, 노선번호 맞추기
  - [X] 서울시 지하철 실시간 도착 정보에 쓰이는 역 종류 확인
  - [X] ~~전체 지하철 시간표 정보에 쓰이는 역 종류 확인~~
  - [X] 역사명 통일
  - [X] 노선명 통일
  - [X] 역번호 통일
  - [ ] ~~노선번호 통일~~
- [X] 노선도와 비교
- [ ] ~~서울 인접 역만 남기기~~
  - [X] 주소 데이터 결측값 확인
  - [X] 결측값 수동으로 채우기
  - [ ] ~~서울 인접 역만 남기고 자르기~~
  - [ ] ~~이상값 확인~~
- [X] 필요 없는 컬럼 제거
- [ ] 환승역, 종점, 분기역 정보 가공
- [X] 파일 생성

### 0. import

In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import koreanize_matplotlib
import folium

### 1. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [115]:
# 데이터 읽어오기
station = pd.read_csv('train_station.csv')

# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['영문역사명', '한자역사명', '운영기관명', '역사전화번호','데이터기준일자'])

station.rename(columns={'역사도로명주소': '주소'}, inplace=True)

# 확실히 필요없는 행 제거
# # 인천 1~2호선
# station = station[(station['노선번호'].str.slice(stop=3) != 'S28')]
# 인천 자기부상철도 -> 휴업 상태
station = station[(station['노선번호'] != 'S28M1')]
# 대전
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '30')]
# 대구 1~3호선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '27')]
# 부산 1~4호선, 김해선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '26') & (station['노선번호'] != 'L48B1')]
# 광주
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '29')]

# # 나중을 위해 역번호 저장
# station_id = station['역번호']

station = station.reset_index(drop=True)
station

,역번호,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역위도,역경도,주소
0,3110,계양,S2801,인천지하철 1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319,인천광역시 계양구 다남로 24
1,3111,귤현,S2801,인천지하철 1호선,일반역,-,-,37.566362,126.742498,인천광역시 계양구 장제로 1136
2,3112,박촌,S2801,인천지하철 1호선,일반역,-,-,37.553525,126.744946,인천광역시 계양구 장제로 992
3,3113,임학,S2801,인천지하철 1호선,일반역,-,-,37.545058,126.738642,인천광역시 계양구 장제로 875
4,3114,계산,S2801,인천지하철 1호선,일반역,-,-,37.543243,126.728436,인천광역시 계양구 경명대로 1089
...,...,...,...,...,...,...,...,...,...,...
748,4811,달미역,I41WS,서해선,일반역,NaN,NaN,37.349320,126.808218,경기도 안산시 단원구 순환로 지하 160
749,4812,선부역,I41WS,서해선,일반역,NaN,NaN,37.333908,126.810977,경기도 안산시 단원구 선부광장로 34
750,4813,초지역,I41WS,서해선,환승역,"I4103, I28K1","수인선, 과천안산선",37.320583,126.806151,경기도 안산시 단원구 중앙대로 620
751,4814,시우역,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303,경기도 안산시 단원구 동산로 지하 50


### 2. 결측값 확인

##### 2-a. 위경도 결측값 분석

In [116]:
# 역이 노선별로 총 몇 개인지 확인
# 1호선이어도, 구간별로 노선 명칭 다름 (우리가 아는 1호선 = 경원선 + 1호선 + 경인선 + 경부선 + 장항선)
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선               10
2호선               50
3호선               34
4호선               26
5호선               56
6호선               39
7호선               42
8호선               18
경강선               11
경부선               45
경원선               24
경의중앙선             57
경인선               20
경춘선               24
김포골드라인            10
도시철도 7호선          11
분당선               37
서해선               12
수도권  도시철도 9호선     25
수도권 경량도시철도 신림선    11
수인선               26
신분당선              16
안산과천선             21
에버라인              15
우이신설선             13
의정부               15
인천국제공항선           14
인천지하철 1호선         30
인천지하철 2호선         27
일산선               11
진접선                3
Name: 노선명, dtype: int64

In [117]:
# 결측값이 컬럼별로 몇 개인지 확인
station.isnull().sum()

역번호         0
역사명         0
노선번호        0
노선명         0
환승역구분       0
환승노선번호    476
환승노선명     476
역위도       352
역경도       352
주소          0
dtype: int64

In [118]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_station_cnt_by_route = station['노선명'][station['역위도'].isnull(
)].value_counts().sort_index()
null_station_cnt_by_route

1호선              10
2호선              50
3호선              34
4호선              26
5호선              56
6호선              39
7호선              42
8호선              18
김포골드라인           10
도시철도 7호선         11
수도권  도시철도 9호선    25
우이신설선            13
의정부              15
진접선               3
Name: 노선명, dtype: int64

##### 2-b. 결측값 채워진 데이터셋 구하기

In [119]:
# 1. 노선별 전체 역 개수가 실제 역 개수와 다름
# 2. 위경도 결측 데이터가 존재하는 노선에 대해 알아본 결과, 서울시 역사마스터 정보와 제대로 합처지지 않은 것으로 보임
# -> 서울시 역사마스터 정보와의 결합 필요

# 서울시 역사마스터 데이터 읽어오기
station_seoul = pd.read_csv('train_station_seoul.csv')

# 기존 데이터셋과 컬럼 이름 맞추기
station_seoul.rename(columns={'역사_ID': '역번호', '호선': '노선명'}, inplace=True)
station.rename(columns={'역위도': '위도', '역경도': '경도'}, inplace=True)

# 서울시 역사마스터 데이터, 위도와 경도가 반대로 들어가 있는 문제 해결
station_seoul.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)
station_seoul = station_seoul[['역번호', '역사명', '노선명', '위도', '경도']]

station_seoul

,역번호,역사명,노선명,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
760,154,종로5가,1호선,37.570926,127.001849
761,153,종로3가,1호선,37.570406,126.991847
762,152,종각,1호선,37.570161,126.982923
763,151,시청,1호선,37.565715,126.977088


In [120]:
# 역이 노선별로 몇 개인지 확인
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

# 확인 결과, 앞서 전체 역정보 데이터셋에 빠진 일부 역이 이 데이터셋에는 포함되어 있음 (역 개수 더 많음)

1호선          10
2호선          50
3호선          34
4호선          26
5호선          58
6호선          39
7호선          53
7호선(인천)       9
8호선          18
9호선          25
9호선(연장)      13
경강선          11
경부선          39
경원선          30
경의중앙선        31
경인선          20
경춘선          19
공항철도1호선      14
과천선           9
김포골드라인       10
분당선          35
서해선          12
수인선          18
신림선          11
신분당선          6
신분당선(연장)      7
신분당선(연장2)     3
안산선          13
에버라인선        15
우이신설선        13
의정부선         15
인천1호선        30
인천2호선        27
일산선          11
장항선           7
중앙선          21
진접선           3
Name: 노선명, dtype: int64

In [121]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
station_seoul.isnull().sum()

# 확인 결과, 해당 데이터셋은 위경도 정보가 빠진 것 없이 잘 들어있음 

역번호    0
역사명    0
노선명    0
위도     0
경도     0
dtype: int64

##### 2-c. 노선명 통일

In [122]:
# 둘 중 한 데이터셋에만 있는 노선명 확인
route_name = set(total_station_cnt_by_route.index)
route_name_seoul = set(total_station_seoul_cnt_by_route.index)
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

{'의정부', '도시철도 7호선', '인천지하철 1호선', '수도권  도시철도 9호선', '인천국제공항선', '인천지하철 2호선', '에버라인', '안산과천선', '수도권 경량도시철도 신림선'}
{'신림선', '과천선', '9호선(연장)', '인천2호선', '인천1호선', '의정부선', '9호선', '중앙선', '공항철도1호선', '장항선', '7호선(인천)', '신분당선(연장)', '신분당선(연장2)', '안산선', '에버라인선'}


In [123]:
# 노선명 통일
station["노선명"] = station["노선명"].replace({
    "도시철도 7호선": "7호선",
    "수도권  도시철도 9호선": "9호선",
    "수도권 경량도시철도 신림선": "신림선",
    "인천지하철 1호선": "인천1호선",
    "인천지하철 2호선": "인천2호선",
    "인천국제공항선": "공항철도선",
    "에버라인": "에버라인선",
    "의정부": "의정부선"
})

station_seoul["노선명"] = station_seoul["노선명"].replace({
    "공항철도1호선": "공항철도선",
    "안산선": "안산과천선",
    "과천선": "안산과천선",
    "9호선(연장)": "9호선",
    "7호선(인천)": "7호선",
    "신분당선(연장)": "신분당선",
    "신분당선(연장2)": "신분당선"
})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name = set(station["노선명"])
route_name_seoul = set(station_seoul["노선명"])
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

# 장항선, 중앙선은 이후 역 종류 보고 어디에 넣을지 결정하기 

set()
{'장항선', '중앙선'}


In [124]:
# 각 노선별 역 개수 확인해 보기
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선       10
2호선       50
3호선       34
4호선       26
5호선       56
6호선       39
7호선       53
8호선       18
9호선       25
경강선       11
경부선       45
경원선       24
경의중앙선     57
경인선       20
경춘선       24
공항철도선     14
김포골드라인    10
분당선       37
서해선       12
수인선       26
신림선       11
신분당선      16
안산과천선     21
에버라인선     15
우이신설선     13
의정부선      15
인천1호선     30
인천2호선     27
일산선       11
진접선        3
Name: 노선명, dtype: int64

In [125]:
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

1호선       10
2호선       50
3호선       34
4호선       26
5호선       58
6호선       39
7호선       62
8호선       18
9호선       38
경강선       11
경부선       39
경원선       30
경의중앙선     31
경인선       20
경춘선       19
공항철도선     14
김포골드라인    10
분당선       35
서해선       12
수인선       18
신림선       11
신분당선      16
안산과천선     22
에버라인선     15
우이신설선     13
의정부선      15
인천1호선     30
인천2호선     27
일산선       11
장항선        7
중앙선       21
진접선        3
Name: 노선명, dtype: int64

##### 2-d. 역사명 통일

In [126]:
# 둘 중 한 데이터셋에만 있는 역사명 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['가능역', '가산디지털단지역', '가좌역', '가천대역', '가평역', '간석역', '갈매역', '강남구청역', '강매역', '강촌역', '개봉역', '개포동역', '걸포북변역', '검단오류', '경기광주역', '경마공원역', '고색역', '고잔역', '고촌역', '곡산역', '곤지암역', '공덕역', '과천역', '관악산', '관악역', '광명역', '광운대역', '교대(법원·검찰청)', '구래역', '구로역', '구룡역', '구리역', '구성역', '구일역', '국수역', '군포역', '굴봉산역', '금곡역', '금릉역', '금정역', '금천구청역', '금촌역', '기흥(백남준아트센터)', '기흥역', '김유정역', '김포공항역', '낙성대(강감찬)', '남동인더스파크역', '남영역', '남춘천역', '남한산성입구(성남법원·검찰청)', '노량진역', '녹양역', '녹천역', '능곡역', '달미역', '달월역', '당정역', '대곡역', '대공원역', '대모산입구역', '대방역', '대성리역', '대야미역', '대화역', '덕계역', '덕소역', '덕정역', '도곡역', '도농역', '도봉산역', '도봉역', '도심역', '도원역', '도화역', '독산역', '동대문역사문화공원(DDP)', '동두천역', '동두천중앙역', '동백(용인세브란스)', '동암역', '동인천역', '동작', '두정역', '디지털미디어시티역', '마두역', '마산역', '마석역', '망우역', '망월사역', '망포역', '매교역', '매탄권선역', '명학역', '모란역', '문산역', '미금역', '반월역', '방학역', '배방역', '백마역', '백석역', '백양리역', '백운역', '범계역', '별내별가람역', '별내역', '병점역', '보산역', '보정역', '복정역', '봉명역', '부개역', '부발역', '부천역', '부평역', '사릉역', '사리역', '사우(김포시청)역', '산본역', '삼동역', '삼송역', '상갈역', '상록수역', '상봉역', '상천역', '서강대

In [127]:
# 역이름 통일
# 역이름 맨 끝에 등장하는'역'자 제거
station['역사명'] = station['역사명'].str.replace(pat=r'역$', repl=r'', regex=True)
# 서울역은 다시 '역' 추가 처리 
station['역사명'] = station['역사명'].str.replace(pat=r'^서울$', repl=r'서울역', regex=True)

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['검단오류', '관악산', '교대(법원·검찰청)', '기흥(백남준아트센터)', '낙성대(강감찬)', '남한산성입구(성남법원·검찰청)', '동대문역사문화공원(DDP)', '동백(용인세브란스)', '동작', '상봉', '서울역(경의선)', '성신여대입구', '수원(분당)', '시민공원', '시우', '시청·용인대', '신촌(지하)', '아시아드경기장', '온수', '왕십리', '용마산(용마폭포공원)', '운동장·송담대(중앙시장)', '운연', '이촌', '전대·에버랜드', '종로3가(탑골공원)', '청량리', '하남시청(덕풍∙신장)', '흑석']
['검단오류(검단산업단지)', '관악산(서울대)', '교대(법원.검찰청)', '낙성대', '남한산성입구(성남법원.검찰청)', '동대문역사문화공원', '동백', '둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '시민공원(문화창작지대)', '시청.용인대', '아시아드경기장(공촌사거리)', '언주', '용마산', '운동장.송담대', '운연(서창)', '원곡', '이수', '전대.에버랜드', '중앙보훈병원', '하남시청(덕풍-신장)', '한성백제', '흑석(중앙대입구)']


In [128]:
# 특수문자 .으로 통일
station['역사명'] = station['역사명'].str.replace(pat=r'[·∙-]', repl=r'.', regex=True)
station_seoul['역사명'] = station_seoul['역사명'].str.replace(
    pat=r'[·∙-]', repl=r'.', regex=True)

# 수동 맞추기 -> 괄호 생기는 방향으로 (단, 지하/지상 정보, 노선명인 경우 제거)
station["역사명"] = station["역사명"].replace({
    "동작": "동작(현충원)",
    "검단오류": "검단오류(검단산업단지)",
    "관악산": "관악산(서울대)",
    "서울역(경의선)": "서울역",
    "시민공원": "시민공원(문화창작지대)",
    "수원(분당)": "수원",
    "신촌(지하)": "신촌",
    "운연": "운연(서창)",
    "흑석": "흑석(중앙대입구)",
    "아시아드경기장": "아시아드경기장(공촌사거리)",
})

station_seoul["역사명"] = station_seoul["역사명"].replace({
    "낙성대": "낙성대(강감찬)",
    "동대문역사문화공원": "동대문역사문화공원(DDP)",
    "동백": "동백(용인세브란스)",
    "운동장.송담대": "운동장.송담대(중앙시장)",
    "용마산": "용마산(용마폭포공원)",
})

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['기흥(백남준아트센터)', '상봉', '성신여대입구', '시우', '온수', '왕십리', '이촌', '종로3가(탑골공원)', '청량리']
['둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '언주', '원곡', '이수', '중앙보훈병원', '한성백제']


### 3. 두 데이터셋 하나로 결합

##### 3-a. 역사명, 노선명 하나로 합치기

In [129]:
# 일단 하나로 합쳐보기
station_result = pd.merge(station, station_seoul, on=['역사명', '노선명'], how='outer')
station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319,인천광역시 계양구 다남로 24,3110.0,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,37.566362,126.742498,인천광역시 계양구 장제로 1136,3111.0,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,37.553525,126.744946,인천광역시 계양구 장제로 992,3112.0,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,37.545058,126.738642,인천광역시 계양구 장제로 875,3113.0,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,37.543243,126.728436,인천광역시 계양구 경명대로 1089,3114.0,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,NaN,옥수,NaN,경원선,NaN,NaN,NaN,NaN,NaN,NaN,1011.0,37.540446,127.018672
820,NaN,한남,NaN,경원선,NaN,NaN,NaN,NaN,NaN,NaN,1010.0,37.529430,127.009169
821,NaN,서빙고,NaN,경원선,NaN,NaN,NaN,NaN,NaN,NaN,1009.0,37.519594,126.988537
822,NaN,이촌(국립중앙박물관),NaN,경원선,NaN,NaN,NaN,NaN,NaN,NaN,1008.0,37.522427,126.973406


In [130]:
# 역번호_x가 null인 값이 노선별로 몇 개 있는지 각각 확인해보기
null_x_station_cnt_by_route = station_result['노선명'][station_result['역번호_x'].isnull(
)].value_counts().sort_index()
null_x_station_cnt_by_route

5호선       1
7호선       1
9호선      13
경부선       1
경원선       7
경인선       1
서해선       1
안산과천선     1
에버라인선     1
우이신설선     1
장항선       7
중앙선      21
Name: 노선명, dtype: int64

In [131]:
# 역번호_y가 null인 값이 노선별로 몇 개 있는지 각각 확인해보기
null_y_station_cnt_by_route = station_result['노선명'][station_result['역번호_y'].isnull(
)].value_counts().sort_index()
null_y_station_cnt_by_route

5호선       1
7호선       1
경부선       7
경원선       1
경의중앙선    30
경인선       1
경춘선       5
분당선       2
서해선       1
수인선       8
에버라인선     1
우이신설선     1
Name: 노선명, dtype: int64

In [132]:
# 5호선
station_result[(station_result['노선명'] == '5호선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기 
station_result.drop(785, inplace=True)
station_result.iloc[303, [10, 11, 12]] = (
    2535.0, 37.57254, 126.99030)

# 결과 확인
# 모든 역사명을 종로3가(탑골공원)로 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "종로3가": "종로3가(탑골공원)"})
station_result[station_result['역사명'] == '종로3가(탑골공원)']

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
165,0130,종로3가(탑골공원),I4101,1호선,도시철도 환승역,I1103+S1105,수도권 광역철도 3호선+수도권 도시철도 5호선,NaN,NaN,서울특별시 종로구 종로 지하129(종로3가),153.0,37.570406,126.991847
229,0329,종로3가(탑골공원),I1103,3호선,도시철도 환승역,S1105+I1101,수도권 도시철도 5호선+수도권 광역철도 1호선,NaN,NaN,서울특별시 종로구 돈화문로 지하30(묘동),319.0,37.571605,126.991791
303,0534,종로3가(탑골공원),S1105,5호선,도시철도 환승역,I1103+I1101,수도권 광역철도 3호선+수도권 광역철도 1호선,NaN,NaN,서울특별시 종로구 돈화문로11길 지하 26(돈의동),2535.0,37.572540,126.990300


In [133]:
# 7호선
station_result[(station_result['노선명'] == '7호선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(784, inplace=True)
station_result.iloc[403, [10, 11, 12]] = (
    2738.0, 37.485196, 126.981605)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "이수": "총신대입구(이수)",
    "총신대입구": "총신대입구(이수)"})
station_result[station_result['역사명'] == '총신대입구(이수)']

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
276,0432,총신대입구(이수),I1104,4호선,도시철도 환승역,S1107,수도권 도시철도 7호선,NaN,NaN,서울특별시 동작구 동작대로 지하117(사당동),432.0,37.486263,126.981989
403,0736,총신대입구(이수),S1107,7호선,도시철도 환승역,I1104,수도권 광역철도 4호선,NaN,NaN,서울특별시 동작구 사당로 지하310(사당동),2738.0,37.485196,126.981605


In [134]:
# 9호선
station_result[(station_result['노선명'] == '9호선') & (station_result['역번호_x'].isnull())]

# 맞출 데이터 없음

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
771,NaN,중앙보훈병원,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4138.0,37.529191,127.148739
772,NaN,둔촌오륜,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4137.0,37.519683,127.137989
773,NaN,올림픽공원(한국체대),NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4136.0,37.516269,127.130288
774,NaN,한성백제,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4135.0,37.516404,127.116503
775,NaN,송파나루,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4134.0,37.510372,127.112216
776,NaN,석촌,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4133.0,37.505208,127.107040
777,NaN,석촌고분,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4132.0,37.502558,127.097033
778,NaN,삼전,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4131.0,37.504738,127.088025
779,NaN,종합운동장,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4130.0,37.511426,127.076275
780,NaN,봉은사,NaN,9호선,NaN,NaN,NaN,NaN,NaN,NaN,4129.0,37.514219,127.060245


In [135]:
# 경부선
print(len(station_result[(station_result['노선명'] == '장항선') & (
    station_result['역번호_x'].isnull())]))
station_result[(station_result['노선명'] == '경부선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 나머지는 장항선과 합치기

7


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
504,1401,봉명,I4101,경부선,일반역,NaN,NaN,36.801578,127.136319,충남 천안시 동남구 차돌로 51(봉명동 20-2),NaN,NaN,NaN
505,1402,쌍용(나사렛대),I4101,경부선,일반역,NaN,NaN,36.793745,127.121363,충청남도 천안시 서북구 쌍용2동 426-3,NaN,NaN,NaN
506,1403,아산,I4101,경부선,일반역,NaN,NaN,36.792195,127.105049,충남 아산시 배방읍 희망로 90(장재리 325-3),NaN,NaN,NaN
507,1404,탕정,I4101,경부선,일반역,NaN,NaN,36.788272,127.080446,충청남도 아산시 탕정면 매곡중앙6로 11,NaN,NaN,NaN
508,1405,배방,I4101,경부선,일반역,NaN,NaN,36.777541,127.052751,충남 아산시 배방읍 온천대로 1967,NaN,NaN,NaN
509,1407,온양온천,I4101,경부선,일반역,NaN,NaN,36.780419,127.003707,충남 아산시 온천대로 1496(온천동 56-9),NaN,NaN,NaN
510,1408,신창(순천향대),I4101,경부선,일반역,NaN,NaN,36.769213,126.950129,충남 아산시 신창면 행목로 50,NaN,NaN,NaN
823,NaN,서울역,NaN,경부선,NaN,NaN,NaN,NaN,NaN,NaN,1001.0,37.554337,126.971134


In [136]:
# 서울역 데이터 수정 필요
station_result.drop([558, 823], inplace=True)
station_result[station_result['역사명'] == '서울역']

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
92,A01,서울역,I28A1,공항철도선,환승역,S1101+S1104+I4108,서울 도시철도 1호선+서울 도시철도 4호선+경의선,37.549118,126.970464,서울특별시 용산구 청파로 378,4201.0,37.553247,126.969769
168,0133,서울역,I4101,1호선,도시철도 환승역,I1104+I41K4+I28A1,수도권 광역철도 4호선+수도권 광역철도 경의중앙+수도권 광역철도 공항,NaN,NaN,서울특별시 중구 세종대로 지하2(남대문로 5가),150.0,37.556228,126.972135
270,0426,서울역,I1104,4호선,도시철도 환승역,I4101+I41K4+I28A1,수도권 광역철도 1호선+수도권 광역철도 경의중앙+수도권 광역철도 공항,NaN,NaN,서울특별시 용산구 한강대로 지하392(동자동),426.0,37.552810,126.972556
559,1251,서울역,I4108,경의중앙선,환승역,"S2601, L2604","1호선, 4호선",37.554713,126.970787,서울시 용산구 한강대로 405,1251.0,37.557231,126.971030


In [137]:
# 장항선
print(len(station_result[(station_result['노선명'] == '장항선') & (
    station_result['역번호_x'].isnull())]))
station_result[(station_result['노선명'] == '장항선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop([788, 789, 790, 791, 792, 793, 794] , inplace=True)
station_result.iloc[504, [10, 11, 12]] = (
    1401.0, 36.801215, 127.135763)
station_result.iloc[505, [10, 11, 12]] = (
    1402.0, 36.793759, 127.121400)
station_result.iloc[506, [10, 11, 12]] = (
    1403.0, 36.792053, 127.104361)
station_result.iloc[507, [10, 11, 12]] = (
    1404.0, 36.788660, 127.084850)
station_result.iloc[508, [10, 11, 12]] = (
    1405.0, 36.777629, 127.052991)
station_result.iloc[509, [10, 11, 12]] = (
    1407.0, 36.780483, 127.003249)
station_result.iloc[510, [10, 11, 12]] = (
    1408.0, 36.769502, 126.951108)

station_result[(station_result['노선명'] == '장항선')]

7


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y


In [138]:
# 바뀐 값 확인
station_result.iloc[504:511]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
504,1401,봉명,I4101,경부선,일반역,NaN,NaN,36.801578,127.136319,충남 천안시 동남구 차돌로 51(봉명동 20-2),1401.0,36.801215,127.135763
505,1402,쌍용(나사렛대),I4101,경부선,일반역,NaN,NaN,36.793745,127.121363,충청남도 천안시 서북구 쌍용2동 426-3,1402.0,36.793759,127.121400
506,1403,아산,I4101,경부선,일반역,NaN,NaN,36.792195,127.105049,충남 아산시 배방읍 희망로 90(장재리 325-3),1403.0,36.792053,127.104361
507,1404,탕정,I4101,경부선,일반역,NaN,NaN,36.788272,127.080446,충청남도 아산시 탕정면 매곡중앙6로 11,1404.0,36.788660,127.084850
508,1405,배방,I4101,경부선,일반역,NaN,NaN,36.777541,127.052751,충남 아산시 배방읍 온천대로 1967,1405.0,36.777629,127.052991
509,1407,온양온천,I4101,경부선,일반역,NaN,NaN,36.780419,127.003707,충남 아산시 온천대로 1496(온천동 56-9),1407.0,36.780483,127.003249
510,1408,신창(순천향대),I4101,경부선,일반역,NaN,NaN,36.769213,126.950129,충남 아산시 신창면 행목로 50,1408.0,36.769502,126.951108


In [139]:
# 경인선
station_result[(station_result['노선명'] == '경인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(786, inplace=True)
station_result.iloc[620, [10, 11, 12]] = (
    1821.0, 37.492433, 126.824086)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "온수": "온수(성공회대입구)"})
station_result[station_result['역사명'] == "온수(성공회대입구)"]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
417,0750,온수(성공회대입구),S1107,7호선,도시철도 환승역,I4101,수도권 광역철도 1호선,NaN,NaN,서울특별시 구로구 경인로3길 지하64(온수동),2752.0,37.492092,126.823023
620,1821,온수(성공회대입구),I1101,경인선,환승역,S1107,7호선,37.492267,126.823399,서울시 구로구 부일로 872,NaN,NaN,NaN


In [140]:
# 경춘선
station_result[(station_result['노선명'] == '경춘선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
637,1015,회기,I41K2,경춘선,환승역,"I4108, I4101","경의중앙선, 경원선",37.589802,127.057936,서울시 동대문구 회기로196(휘경동 317-101),NaN,NaN,NaN
638,1014,청량리,I41K2,경춘선,환승역,"S2601, I4108, I28K1","1호선, 경의중앙선, 분당선",37.580543,127.046516,서울시 동대문구 왕산로 214,NaN,NaN,NaN
639,1201,중랑,I41K2,경춘선,환승역,I4108,경의중앙선,37.594948,127.076152,서울특별시 중랑구 중화동 321-37,NaN,NaN,NaN
640,1202,상봉,I41K2,경춘선,환승역,"S1107, I4108","7호선, 경의중앙선",37.595254,127.085853,서울시 중랑구 망우로 297(상봉1동 100-9번지),NaN,NaN,NaN
641,1203,망우,I41K2,경춘선,환승역,I4108,경의중앙선,37.599288,127.092318,서울시 중랑구 망우로 55길 11-10,NaN,NaN,NaN


In [141]:
# 분당선
station_result[(station_result['노선명'] == '분당선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
661,1014,청량리,I28K1,분당선,환승역,"S2601, I41K2, I4108","1호선, 경춘선, 경의중앙선",37.580543,127.046516,서울시 동대문구 왕산로 214,NaN,NaN,NaN
662,1013,왕십리,I28K1,분당선,환승역,"S2602, S1105, I4108","2호선, 5호선, 경의중앙선",37.561128,127.035505,서울시 성동구 왕십리 광장로 17(행당동),NaN,NaN,NaN


In [142]:
# 서해선
station_result[(station_result['노선명'] == '서해선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 원곡역 -> 시우역으로 이름 변화함
station_result.drop(768, inplace=True)
station_result.iloc[766, [10, 11, 12]] = (
    4814.0, 37.31321, 126.796261)

station_result[(station_result['노선명'] == '서해선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
766,4814,시우,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303,경기도 안산시 단원구 동산로 지하 50,NaN,NaN,NaN


In [143]:
# 안산과천선
station_result[(station_result['노선명'] == '안산과천선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
787,NaN,금정,NaN,안산과천선,NaN,NaN,NaN,NaN,NaN,NaN,1458.0,37.372209,126.943417


In [144]:
# 수인선
station_result[(station_result['노선명'] == '수인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
703,1755,한대앞,I28K1,수인선,일반역,NaN,NaN,37.309685,126.853656,경기도 안산시 상록구 충장로 337,NaN,NaN,NaN
704,1756,중앙,I28K1,수인선,일반역,NaN,NaN,37.316055,126.838532,경기도 안산시 단원구 중앙대로 918,NaN,NaN,NaN
705,1757,고잔,I28K1,수인선,일반역,NaN,NaN,37.316821,126.823094,경기도 안산시 상록구 충장로 337,NaN,NaN,NaN
706,1758,초지,I28K1,수인선,환승역,"I4103, I41WS","안산과천선, 서해선",37.320583,126.806151,경기도 안산시 단원구 중앙대로 620,NaN,NaN,NaN
707,1759,안산,I28K1,수인선,일반역,NaN,NaN,37.327006,126.788748,경기도 안산시 단원구 중앙대로 620,NaN,NaN,NaN
708,1760,신길온천,I28K1,수인선,일반역,NaN,NaN,37.337488,126.767306,경기도 안산시 단원구 중앙대로 462,NaN,NaN,NaN
709,1761,정왕,I28K1,수인선,일반역,NaN,NaN,37.351816,126.742844,경기도 안산시 단원구 황고개로 2,NaN,NaN,NaN
710,1762,오이도,I28K1,수인선,일반역,NaN,NaN,37.361773,126.738437,경기도 시흥시 역전로 430,NaN,NaN,NaN


In [145]:
# 에버라인선
station_result[(station_result['노선명'] == '에버라인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(770, inplace=True)
station_result.iloc[77, [10, 11, 12]] = (
    4501.0, 37.275449, 127.116665)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "기흥": "기흥(백남준아트센터)"})
station_result[station_result['역사명'] == "기흥(백남준아트센터)"]


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
77,Y110,기흥(백남준아트센터),L41E1,에버라인선,환승역,L41E1,수도권 경량도시철도 에버라인,37.275453,127.117046,경기도 용인시 기흥구 중부대로 460,4501.0,37.275449,127.116665
689,1865,기흥(백남준아트센터),I28K1,분당선,환승역,L41E1,용인경전철,37.275657,127.115944,경기도 용인시 중부대로 460,1865.0,37.275061,127.115910


In [146]:
# 우이신설선
station_result[(station_result['노선명'] == '우이신설선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(769, inplace=True)
station_result.iloc[116, [10, 11, 12]] = (
    4711.0, 37.592467, 127.016516)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "성신여대입구": "성신여대입구(돈암)",
    "돈암": "성신여대입구(돈암)"})
station_result[station_result['역사명'] == "성신여대입구(돈암)"]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
116,S120,성신여대입구(돈암),L11UI,우이신설선,도시철도 환승역,I1104,수도권 광역철도 4호선,NaN,NaN,서울특별시 성북구 동선동1가 116-1,4711.0,37.592467,127.016516
262,0418,성신여대입구(돈암),I1104,4호선,도시철도 일반역,I1104,수도권 광역철도 4호선,NaN,NaN,서울특별시 성북구 동소문로 지하102(동선동4가),418.0,37.592612,127.016441


In [147]:
# 경원선
print(len(station_result[(station_result['노선명'] == '경원선') & (
    station_result['역번호_x'].isnull())]))
station_one = station_result[(station_result['노선명'] == '경원선') & (
    station_result['역번호_x'].isnull())]

station_one = station_one.replace({'경원선': '경의중앙선'})

station_one

7


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
816,NaN,청량리(서울시립대입구),NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1014.0,37.580759,127.048300
817,NaN,왕십리(성동구청),NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1013.0,37.561827,127.038352
818,NaN,응봉,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1012.0,37.549946,127.034538
819,NaN,옥수,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1011.0,37.540446,127.018672
820,NaN,한남,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1010.0,37.529430,127.009169
821,NaN,서빙고,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1009.0,37.519594,126.988537
822,NaN,이촌(국립중앙박물관),NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1008.0,37.522427,126.973406


In [148]:
# 중앙선
print(len(station_result[(station_result['노선명'] == '중앙선') & (
    station_result['역번호_x'].isnull())]))
station_cau = station_result[(station_result['노선명'] == '중앙선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

station_cau = station_cau.replace({'중앙선': '경의중앙선'})

station_cau

21


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
795,NaN,지평,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1220.0,37.476393,127.629874
796,NaN,용문,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1219.0,37.482230,127.594647
797,NaN,원덕,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1218.0,37.468672,127.547076
798,NaN,양평,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1217.0,37.492773,127.491837
799,NaN,오빈,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1216.0,37.506062,127.473868
800,NaN,아신,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1215.0,37.513820,127.443173
801,NaN,국수,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1214.0,37.516169,127.399367
802,NaN,신원,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1213.0,37.525545,127.372921
803,NaN,양수,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1212.0,37.545981,127.329098
804,NaN,운길산,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,NaN,1211.0,37.554669,127.310115


In [149]:
# 경의중앙선
print(len(station_result[(station_result['노선명'] == '경의중앙선') & (
    station_result['역번호_y'].isnull())]))
station_gyeong = station_result[(station_result['노선명'] == '경의중앙선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# y 컬럼 제거
station_gyeong = station_gyeong.drop(columns=['역번호_y', '위도_y', '경도_y'])

# 역이름 변경
station_gyeong = station_gyeong.replace({
    '청량리': '청량리(서울시립대입구)',
    '왕십리': '왕십리(성동구청)',
    '이촌': '이촌(국립중앙박물관)',
    '상봉': '상봉(시외버스터미널)'
})

station_result = station_result.replace({
    '청량리': '청량리(서울시립대입구)',
    '왕십리': '왕십리(성동구청)',
    '이촌': '이촌(국립중앙박물관)',
    '상봉': '상봉(시외버스터미널)'
})

station_gyeong

30


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소
562,1003,용산,I4108,경의중앙선,환승역,I4101,경부선,37.529890,126.964775,서울시 용산구 한강대로 23길 55(한강로 3가 40-999)
582,1013,왕십리(성동구청),I4108,경의중앙선,환승역,"S2602, S1105, I28K1","2호선, 5호선, 분당선",37.561128,127.035505,서울시 성동구 왕십리 광장로 17(행당동)
583,1014,청량리(서울시립대입구),I4108,경의중앙선,환승역,"S2601, I41K2, I28K1","1호선, 경춘선, 분당선",37.580543,127.046516,서울시 동대문구 왕산로 214
584,1015,회기,I4108,경의중앙선,환승역,"I4101, I41K2","경원선, 경춘선",37.589802,127.057936,서울시 동대문구 회기로196(휘경동 317-101)
585,1201,중랑,I4108,경의중앙선,환승역,I41K2,경춘선,37.594948,127.076152,서울특별시 중랑구 중화동 321-37
586,1202,상봉(시외버스터미널),I4108,경의중앙선,환승역,"S1107, I41K2","7호선, 경춘선",37.595254,127.085853,서울시 중랑구 망우로 297(상봉1동 100-9번지)
587,1203,망우,I4108,경의중앙선,환승역,I41K2,경춘선,37.599288,127.092318,서울시 중랑구 망우로 55길 11-10
588,1204,양원,I4108,경의중앙선,일반역,NaN,NaN,36.963209,129.091131,서울시 중랑구 송림길 147(망우동)
589,1220,지평,I4108,경의중앙선,일반역,NaN,NaN,37.476496,127.629681,경기도 양평군 지평면 지평역길 32
590,1205,구리,I4108,경의중앙선,일반역,NaN,NaN,37.503178,126.882037,경기도 구리시 건원대로 34번길 32-29(인창동 244-2)


In [150]:
# 경원선 + 중앙선 = 경의중앙선 으로 합치기
station_temp = pd.concat([station_one, station_cau], axis = 0)

# x 컬럼 제거
station_temp = station_temp.drop(columns=['역번호_x', '노선번호', '환승역구분', '환승노선번호', '환승노선명', '주소', '위도_x', '경도_x'])

# merge
station_temp = pd.merge(station_gyeong, station_temp, on=['역사명', '노선명'], how='outer')

station_temp

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
0,1003,용산,I4108,경의중앙선,환승역,I4101,경부선,37.529890,126.964775,서울시 용산구 한강대로 23길 55(한강로 3가 40-999),NaN,NaN,NaN
1,1013,왕십리(성동구청),I4108,경의중앙선,환승역,"S2602, S1105, I28K1","2호선, 5호선, 분당선",37.561128,127.035505,서울시 성동구 왕십리 광장로 17(행당동),1013.0,37.561827,127.038352
2,1014,청량리(서울시립대입구),I4108,경의중앙선,환승역,"S2601, I41K2, I28K1","1호선, 경춘선, 분당선",37.580543,127.046516,서울시 동대문구 왕산로 214,1014.0,37.580759,127.048300
3,1015,회기,I4108,경의중앙선,환승역,"I4101, I41K2","경원선, 경춘선",37.589802,127.057936,서울시 동대문구 회기로196(휘경동 317-101),1015.0,37.589460,127.057583
4,1201,중랑,I4108,경의중앙선,환승역,I41K2,경춘선,37.594948,127.076152,서울특별시 중랑구 중화동 321-37,1201.0,37.594917,127.076116
5,1202,상봉(시외버스터미널),I4108,경의중앙선,환승역,"S1107, I41K2","7호선, 경춘선",37.595254,127.085853,서울시 중랑구 망우로 297(상봉1동 100-9번지),1202.0,37.596678,127.085040
6,1203,망우,I4108,경의중앙선,환승역,I41K2,경춘선,37.599288,127.092318,서울시 중랑구 망우로 55길 11-10,1203.0,37.599550,127.091909
7,1204,양원,I4108,경의중앙선,일반역,NaN,NaN,36.963209,129.091131,서울시 중랑구 송림길 147(망우동),1204.0,37.606596,127.107906
8,1220,지평,I4108,경의중앙선,일반역,NaN,NaN,37.476496,127.629681,경기도 양평군 지평면 지평역길 32,1220.0,37.476393,127.629874
9,1205,구리,I4108,경의중앙선,일반역,NaN,NaN,37.503178,126.882037,경기도 구리시 건원대로 34번길 32-29(인창동 244-2),1205.0,37.603392,127.143869


In [151]:
# 기존 데이터에 추가, 기존에 있던 애들은 삭제
station_result.drop(index=station_one.index, inplace=True)
station_result.drop(index=station_cau.index, inplace=True)
station_result.drop(index=station_gyeong.index, inplace=True)
station_result = pd.concat([station_result, station_temp])

station_result = station_result.reset_index(drop=True)
station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319,인천광역시 계양구 다남로 24,3110.0,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,37.566362,126.742498,인천광역시 계양구 장제로 1136,3111.0,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,37.553525,126.744946,인천광역시 계양구 장제로 992,3112.0,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,37.545058,126.738642,인천광역시 계양구 장제로 875,3113.0,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,37.543243,126.728436,인천광역시 계양구 경명대로 1089,3114.0,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,1008,이촌(국립중앙박물관),I4108,경의중앙선,환승역,L2604,4호선,37.522476,126.973816,서울시 용산구 이촌로 71길 42,1008.0,37.522427,126.973406
777,1009,서빙고,I4108,경의중앙선,일반역,NaN,NaN,37.519610,126.988358,서울시 용산구 서빙고로 238(서빙고동 241-11),1009.0,37.519594,126.988537
778,1010,한남,I4108,경의중앙선,일반역,NaN,NaN,37.529361,127.008984,서울시 용산구 독서당로6길 12-13(한남동),1010.0,37.529430,127.009169
779,1011,옥수,I4108,경의중앙선,환승역,S2603,3호선,37.540514,127.018698,서울시 성동구 동호로 지하21(옥수동),1011.0,37.540446,127.018672


##### 3-b. 노선명을 지하철 호선명으로 통일, 중복값 확인

In [152]:
# 노선명 종류 확인
station_cnt_by_route = station_result['노선명'].value_counts().sort_index()
station_cnt_by_route

1호선       10
2호선       50
3호선       34
4호선       26
5호선       58
6호선       39
7호선       62
8호선       18
9호선       38
경강선       11
경부선       45
경원선       24
경의중앙선     60
경인선       20
경춘선       24
공항철도선     14
김포골드라인    10
분당선       37
서해선       12
수인선       26
신림선       11
신분당선      16
안산과천선     22
에버라인선     15
우이신설선     13
의정부선      15
인천1호선     30
인천2호선     27
일산선       11
진접선        3
Name: 노선명, dtype: int64

In [153]:
# 경부선, 경원선, 경인선 = 1호선
# 분당선 + 수인선 = 수인분당선
# 일산선 = 3호선
# 진접선 = 4호선

station_result["노선명"] = station_result["노선명"].replace({
    "경부선": "1호선",
    "경원선": "1호선",
    "경인선": "1호선",
    "분당선": "수인분당선",
    "수인선": "수인분당선",
    "일산선": "3호선",
    "안산과천선": "4호선", 
    "진접선": "4호선"
})

station_result["노선명"] = station_result["노선명"].replace({
    "수인수인분당선": "수인분당선"
})

# 노선명 종류 재확인
station_cnt_by_route = station_result['노선명'].value_counts().sort_index()
station_cnt_by_route

1호선       99
2호선       50
3호선       45
4호선       51
5호선       58
6호선       39
7호선       62
8호선       18
9호선       38
경강선       11
경의중앙선     60
경춘선       24
공항철도선     14
김포골드라인    10
서해선       12
수인분당선     63
신림선       11
신분당선      16
에버라인선     15
우이신설선     13
의정부선      15
인천1호선     30
인천2호선     27
Name: 노선명, dtype: int64

In [154]:
# 노선별 중복 확인
station_result[station_result.duplicated(['노선명', '역사명'])]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
58,3753,까치울,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 626 (춘의동),2753.0,37.506207,126.810939
60,3754,부천종합운동장,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 502 (춘의동),2754.0,37.505380,126.797337
62,3755,춘의,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 406 (춘의동),2755.0,37.503663,126.787036
64,3756,신중동,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 314 (중동),2756.0,37.503048,126.775960
66,3757,부천시청,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 202 (중동),2757.0,37.504631,126.763538
68,3758,상동,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 104 (상동),2758.0,37.505781,126.753083
70,3759,삼산체육관,S1107,7호선,일반역,-,-,NaN,NaN,인천광역시 부평구 길주로 지하 713 (삼산동),2759.0,37.506411,126.742153
72,3760,굴포천,S1107,7호선,일반역,-,-,NaN,NaN,인천광역시 부평구 길주로 지하 623 (삼산동),2760.0,37.506997,126.731280
74,3761,부평구청,S1107,7호선,환승역,S1107+S2801,인천1호선,NaN,NaN,인천광역시 부평구 길주로 지하 527 (갈산동),2761.0,37.507394,126.721599
330,0554,강일,S1105,5호선,도시철도 일반역,NaN,NaN,NaN,NaN,서울특별시 강동구 고덕로 지하456(강일동),2562.0,37.557490,127.175930


In [155]:
# 7호선 데이터 확인
station_result.iloc[57:75]

# 역번호_y, 위도, 경도 정보가 서로 달라 잘못 저장됨
# 확인 결과, 2로 시작하는 쪽의 역번호가 틀림 (짝수 index 행들)

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
57,3753,까치울,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 626 (춘의동),3753.0,37.506130,126.810930
58,3753,까치울,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 626 (춘의동),2753.0,37.506207,126.810939
59,3754,부천종합운동장,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 502 (춘의동),3754.0,37.505020,126.796610
60,3754,부천종합운동장,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 502 (춘의동),2754.0,37.505380,126.797337
61,3755,춘의,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 406 (춘의동),3755.0,37.503650,126.788280
62,3755,춘의,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 406 (춘의동),2755.0,37.503663,126.787036
63,3756,신중동,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 314 (중동),3756.0,37.502820,126.775660
64,3756,신중동,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 314 (중동),2756.0,37.503048,126.775960
65,3757,부천시청,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 202 (중동),3757.0,37.504440,126.763640
66,3757,부천시청,S1107,7호선,일반역,-,-,NaN,NaN,경기도 부천시 원미구 길주로 지하 202 (중동),2757.0,37.504631,126.763538


In [156]:
# 5호선 데이터 확인
station_result.iloc[329:333]

# 역번호, 위도, 경도 정보가 달라 잘못 저장됨
# 확인 결과, 9로 시작하는 쪽의 역번호가 틀림 (홀수 index 행들)

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
329,0554,강일,S1105,5호선,도시철도 일반역,NaN,NaN,NaN,NaN,서울특별시 강동구 고덕로 지하456(강일동),9995.0,37.557490,127.175930
330,0554,강일,S1105,5호선,도시철도 일반역,NaN,NaN,NaN,NaN,서울특별시 강동구 고덕로 지하456(강일동),2562.0,37.557490,127.175930
331,0555,미사,S1105,5호선,도시철도 일반역,NaN,NaN,NaN,NaN,경기도 하남시 미사강변동로 지하90(망월동),9996.0,37.560927,127.193877
332,0555,미사,S1105,5호선,도시철도 일반역,NaN,NaN,NaN,NaN,경기도 하남시 미사강변동로 지하90(망월동),2563.0,37.560927,127.193877


In [157]:
# 경의중앙선 데이터 확인
station_result[(station_result["역사명"] == "공덕") |
               (station_result["역사명"] == "홍대입구") | (station_result["역사명"] == "디지털미디어시티")]

# 확인 결과, 어차피 경중선 역코드는 필요 없음... 1-8호선이 아니므로 
# 그냥 같은 쪽으로 일치시키는 방향으로 변화시키기 

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
93,A02,공덕,I28A1,공항철도선,환승역,S1105+S1106+I41K4,서울 도시철도 5호선+서울 도시철도 6호선+경의중앙선,37.542841,126.951330,서울특별시 마포구 마포대로 92,4202.0,37.542530,126.952024
94,A03,홍대입구,I28A1,공항철도선,환승역,S1102+I41K4,서울 도시철도 2호선+경의중앙선,37.557741,126.926494,서울특별시 마포구 양화로 188,4203.0,37.557438,126.926715
95,A04,디지털미디어시티,I28A1,공항철도선,환승역,S1106+I41K4,서울 도시철도 6호선+경의중앙선,37.576825,126.899021,서울특별시 마포구 성암로 184,4204.0,37.576958,126.898609
207,0239,홍대입구,S1102,2호선,도시철도 환승역,I28A1+I41K4,수도권 광역철도 공항+수도권 광역철도 경의중앙,NaN,NaN,서울특별시 마포구 양화로 지하160(동교동),239.0,37.556790,126.923708
298,0529,공덕,S1105,5호선,도시철도 환승역,S1106,수도권 도시철도 6호선,NaN,NaN,서울특별시 마포구 마포대로 지하100(공덕동),2530.0,37.544431,126.951372
345,0618,디지털미디어시티,S1106,6호선,도시철도 환승역,I41K4+I28A1,수도권 광역철도 경의중앙+수도권 광역철도 공항,NaN,NaN,서울특별시 은평구 수색로 지하175(증산동),2619.0,37.576108,126.901391
353,0626,공덕,S1106,6호선,도시철도 환승역,S1105,수도권 도시철도 5호선,NaN,NaN,서울특별시 마포구 백범로 지하200(공덕동),2627.0,37.543555,126.951678
562,1262,공덕,I4108,경의중앙선,환승역,"S1105, S1106, I28A1","5호선, 6호선, 공항",37.542611,126.952134,서울시 마포구 공덕동 마포대로 자하100,1292.0,37.542596,126.952099
563,1262,공덕,I4108,경의중앙선,환승역,"S1105, S1106, I28A1","5호선, 6호선, 공항",37.542611,126.952134,서울시 마포구 공덕동 마포대로 자하100,1262.0,37.542596,126.952099
565,239,홍대입구,I4108,경의중앙선,환승역,"S2602, I28A1","2호선, 공항",37.558080,126.925533,서울시 마포구 양화로 지하 188,1293.0,37.557641,126.926683


In [158]:
# 3호선 데이터 확인
station_result[(station_result["역사명"] == "지축")]

# 확인 결과, 위쪽이 맞음 

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y
219,0319,지축,I1103,3호선,도시철도 일반역,NaN,NaN,NaN,NaN,경기도 고양시 덕양구 삼송로 300(지축동),309.0,37.648033,126.913917
714,1949,지축,I4106,3호선,일반역,NaN,NaN,37.648114,126.913746,경기도 고양시 덕양구 삼송로 300(지축동),1949.0,37.648017,126.913970


In [159]:
station_result.drop([58, 60, 62, 64, 66, 68, 70, 72, 74, 329, 331, 562, 565, 567, 714], inplace=True)

# 재확인
station_result[station_result.duplicated(['노선명', '역사명'])]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y


##### 3-c. 위경도값 하나로 합치기

In [160]:
# x, y에 대해, 둘 중 한 곳에 없는 위경도값이 각각 몇 개인지 count
lat_in_only_x = station_result[station_result["위도_y"].isnull(
) & station_result["위도_x"].notnull()]
lng_in_only_x = station_result[station_result["경도_y"].isnull(
) & station_result["경도_x"].notnull()]

lat_in_only_x_cnt = len(lat_in_only_x)
lng_in_only_x_cnt = len(lng_in_only_x)

print("x에만 있는 위경도값 개수 :", lat_in_only_x_cnt, lng_in_only_x_cnt)

lat_in_only_y = station_result[station_result["위도_x"].isnull(
) & station_result["위도_y"].notnull()]
lng_in_only_y = station_result[station_result["경도_x"].isnull(
) & station_result["경도_y"].notnull()]

lat_in_only_y_cnt = len(lat_in_only_y)
lng_in_only_y_cnt = len(lng_in_only_y)

print("y에만 있는 위경도값 개수 :", lat_in_only_y_cnt, lng_in_only_y_cnt)

# 한쪽 위경도가 NaN이면, 반대쪽 위경도 넣어주기
station_result["위도_x"] = station_result["위도_x"].fillna(station_result["위도_y"])
station_result["위도_y"] = station_result["위도_y"].fillna(station_result["위도_x"])
station_result["경도_x"] = station_result["경도_x"].fillna(station_result["경도_y"])
station_result["경도_y"] = station_result["경도_y"].fillna(station_result["경도_x"])

# 위경도 데이터의 오차 계산
station_result["위도차"] = (station_result["위도_x"] - station_result["위도_y"]).abs()
station_result["경도차"] = (station_result["경도_x"] - station_result["경도_y"]).abs()

x에만 있는 위경도값 개수 : 20 20
y에만 있는 위경도값 개수 : 366 366


In [161]:
# 위도 오차값 확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff

1.8779144088977806


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y,위도차,경도차
119,D004,신사,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.019761,37.516125,서울특별시 강남구 강남대로 620-2,4304.0,37.516334,127.020114,89.503427,89.503989
120,D005,논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.021366,37.511115,서울특별시 강남구 강남대로 492-2,4305.0,37.511093,127.021415,89.510273,89.510300
121,D006,신논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.024531,37.504555,서울특별시 강남구 강남대로 302-2,4306.0,37.504598,127.025060,89.519933,89.520505
122,D007,강남,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.027879,37.497385,서울특별시 강남구 강남대로 지하 396,4307.0,37.496837,127.028104,89.531042,89.530719
123,D008,양재(서초구청),I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.034099,37.484454,서울시 서초구 남부순환로 지하 2585,4308.0,37.483809,127.034653,89.550290,89.550199
124,D009,양재시민의숲(매헌),I11D1,신분당선,도시철도 일반역,NaN,NaN,127.038420,37.470964,서울특별시 서초구 매헌로 지하 116,4309.0,37.470023,127.038420,89.568397,89.567456
125,D010,청계산입구,I11D1,신분당선,도시철도 일반역,NaN,NaN,127.054340,37.448715,서울특별시 서초구 청계산로 지하 179,4310.0,37.447211,127.055664,89.607129,89.606949
126,D011,판교,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.111211,37.395715,경기도 성남시 분당구 판교역로 지하 160,4311.0,37.394761,127.112217,89.716450,89.716502
127,D012,정자,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.108386,37.368210,경기도 성남시 분당구 성남대로 지하 333,4312.0,37.367098,127.108403,89.741288,89.740193
128,D013,미금,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.108903,37.351183,경기도 성남시 성남대로 지하 163 (금곡동),4313.0,37.349982,127.108918,89.758921,89.757735


In [162]:
# 경도 오차값 확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff

1.8799156203456915


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y,위도차,경도차
54,3225,남동구청,S2802,인천2호선,일반역,-,-,37.448179,123.736581,인천광역시 남동구 인주대로 지하 889 (만수동),3225.0,37.448161,126.736939,0.000018,3.000358
119,D004,신사,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.019761,37.516125,서울특별시 강남구 강남대로 620-2,4304.0,37.516334,127.020114,89.503427,89.503989
120,D005,논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.021366,37.511115,서울특별시 강남구 강남대로 492-2,4305.0,37.511093,127.021415,89.510273,89.510300
121,D006,신논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.024531,37.504555,서울특별시 강남구 강남대로 302-2,4306.0,37.504598,127.025060,89.519933,89.520505
122,D007,강남,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.027879,37.497385,서울특별시 강남구 강남대로 지하 396,4307.0,37.496837,127.028104,89.531042,89.530719
123,D008,양재(서초구청),I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.034099,37.484454,서울시 서초구 남부순환로 지하 2585,4308.0,37.483809,127.034653,89.550290,89.550199
124,D009,양재시민의숲(매헌),I11D1,신분당선,도시철도 일반역,NaN,NaN,127.038420,37.470964,서울특별시 서초구 매헌로 지하 116,4309.0,37.470023,127.038420,89.568397,89.567456
125,D010,청계산입구,I11D1,신분당선,도시철도 일반역,NaN,NaN,127.054340,37.448715,서울특별시 서초구 청계산로 지하 179,4310.0,37.447211,127.055664,89.607129,89.606949
126,D011,판교,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.111211,37.395715,경기도 성남시 분당구 판교역로 지하 160,4311.0,37.394761,127.112217,89.716450,89.716502
127,D012,정자,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.108386,37.368210,경기도 성남시 분당구 성남대로 지하 333,4312.0,37.367098,127.108403,89.741288,89.740193


In [163]:
# 119 ~ 134 위도_x와 경도_x가 뒤바뀌어 들어감
# 송도, (위도_x, 경도_x) 전남 해남에 위치
# 남동구청, (위도_x, 경도_x) 황해에 위치
# 양원, (위도_x, 경도_x) 양원역 위치가 경북 봉화군에 위치한 양원역 위치로 잘못 들어가져 있음
# -> 위경도 데이터의 경우, x를 y로 덮어쓰면 될 것 같음
# 이를 증명하기 위해, 위의 데이터들의 위도차/경도차가 해결된 뒤,
# 위경도차가 높은 행들을 각각 확인하여, 거리상의 오차가 어느 정도일지 확인해볼 것

station_result.loc[[119, 120, 121, 122, 123, 124, 125, 126,
                    127, 128, 129, 130, 131, 132, 133, 134, 688, 54, 758], ['위도차', '경도차']] = 0.0

In [164]:
# 위도 오차값 평균 재확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff.sort_values('위도차').tail()

0.00032537391083540304


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y,위도차,경도차
92,A01,서울역,I28A1,공항철도선,환승역,S1101+S1104+I4108,서울 도시철도 1호선+서울 도시철도 4호선+경의선,37.549118,126.970464,서울특별시 용산구 청파로 378,4201.0,37.553247,126.969769,0.004129,0.000695
736,4815,원시,I41WS,서해선,일반역,NaN,NaN,37.305947,126.788297,경기도 안산시 단원구 선단로 지하 70,4814.0,37.313210,126.796261,0.007263,0.007964
590,1803,역곡,I1101,1호선,일반역,NaN,NaN,37.485140,126.811622,경기도 부천시 원미구 역곡로 1(역곡동),1821.0,37.492433,126.824086,0.007293,0.012464
570,1268,화전,I4108,경의중앙선,일반역,NaN,NaN,37.637837,126.832503,경기도 고양시 덕양구 화랑로 53(화전동 183-10),1268.0,37.602888,126.868387,0.034949,0.035884
760,1205,구리,I4108,경의중앙선,일반역,NaN,NaN,37.503178,126.882037,경기도 구리시 건원대로 34번길 32-29(인창동 244-2),1205.0,37.603392,127.143869,0.100214,0.261832


In [165]:
# 경도 오차값 평균 재확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff.sort_values('경도차').tail()

0.0005800957242818426


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y,위도차,경도차
736,4815,원시,I41WS,서해선,일반역,NaN,NaN,37.305947,126.788297,경기도 안산시 단원구 선단로 지하 70,4814.0,37.313210,126.796261,0.007263,0.007964
590,1803,역곡,I1101,1호선,일반역,NaN,NaN,37.485140,126.811622,경기도 부천시 원미구 역곡로 1(역곡동),1821.0,37.492433,126.824086,0.007293,0.012464
105,A11,인천공항2터미널,I28A1,공항철도선,일반역,NaN,NaN,37.458269,126.427536,인천광역시 중구 공항로 272,4215.0,37.460699,126.441442,0.002430,0.013906
570,1268,화전,I4108,경의중앙선,일반역,NaN,NaN,37.637837,126.832503,경기도 고양시 덕양구 화랑로 53(화전동 183-10),1268.0,37.602888,126.868387,0.034949,0.035884
760,1205,구리,I4108,경의중앙선,일반역,NaN,NaN,37.503178,126.882037,경기도 구리시 건원대로 34번길 32-29(인창동 244-2),1205.0,37.603392,127.143869,0.100214,0.261832


##### 3-b. 위경도 결측값 재확인

In [166]:
# 구리, (위도_x, 경도_x) 구로 들어가 있음
# 화전, (위도_x, 경도_x) 덕양구청에 위치, 도보 1시간 30분의 오차
# 그 다음으로 오차가 큰 인천공항2터미널과 서울역의 경우 역이 워낙 커서 생기는 일, 문제 없음
# 즉, x와 y 두 곳에 있는 값들에 대해서는 이상값 검토가 끝남 
# 둘 다에 있는 값의 경우 그냥 y쪽 값으로 덮어쓰기
# 둘 중 한 곳에만 있는 위경도값들은 위에서 따로 저장해 둠, 
# 이후 지도를 보며 재검토할 예정  

# y에 없는 값들 x에서 가져와 채우기
station_result['위도_y'] = station_result['위도_y'].fillna(station_result['위도_x'])
station_result['경도_y'] = station_result['경도_y'].fillna(station_result['경도_x'])

# 개수 재확인
null_lat_in_y = station_result[station_result["위도_y"].isnull()]
null_lng_in_y = station_result[station_result["경도_y"].isnull()]
null_lat_in_y_cnt = len(null_lat_in_y)
null_lng_in_y_cnt = len(null_lng_in_y)

print("y에 있는 위경도 결측값 개수 :", null_lat_in_y_cnt, null_lng_in_y_cnt)

null_lat_in_y 

y에 있는 위경도 결측값 개수 : 0 0


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,주소,역번호_y,위도_y,경도_y,위도차,경도차


In [167]:
# 위도_x, 경도_x 삭제 및 위도_y와 경도_y의 이름 변경
station_result.drop(columns=["위도_x", "경도_x", "위도차", "경도차"], inplace=True)
station_result.rename(columns={'위도_y': '위도', '경도_y': '경도'}, inplace=True)

station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,인천광역시 계양구 다남로 24,3110.0,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 1136,3111.0,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 992,3112.0,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 875,3113.0,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,인천광역시 계양구 경명대로 1089,3114.0,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...,...
776,1008,이촌(국립중앙박물관),I4108,경의중앙선,환승역,L2604,4호선,서울시 용산구 이촌로 71길 42,1008.0,37.522427,126.973406
777,1009,서빙고,I4108,경의중앙선,일반역,NaN,NaN,서울시 용산구 서빙고로 238(서빙고동 241-11),1009.0,37.519594,126.988537
778,1010,한남,I4108,경의중앙선,일반역,NaN,NaN,서울시 용산구 독서당로6길 12-13(한남동),1010.0,37.529430,127.009169
779,1011,옥수,I4108,경의중앙선,환승역,S2603,3호선,서울시 성동구 동호로 지하21(옥수동),1011.0,37.540446,127.018672


### 4. 지도에 시각화

##### 4-a. 이상값 확인

In [168]:
# 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=0, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']) + \
        " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']
    
    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name, 
        tooltip=popup_name
    ).add_to(fmap)

fmap

In [169]:
# 위경도가 한국을 벗어난 역들 찾아보기
station_outof_korea = station_result[(station_result['위도'] < 33.10000000) |
                                     (station_result['위도'] > 38.45000000) | (station_result['경도'] < 124.19583333) |
                                     (station_result['경도'] > 131.87222222)]

station_outof_korea

# 확인 결과, 모든 역이 제대로 한국 내에 존재함

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도


In [170]:
# 더 크게 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=8, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']
                                                            ) + " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']

    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name,
        tooltip=popup_name
    ).add_to(fmap)

fmap

### 5. 역번호, 노선번호, 역이름, 노선명 맞추기

##### 5-a. 서울시 지하철 실시간 도착정보에 쓰이는 역 및 노선 종류 확인

In [171]:
# 데이터 읽어오기
station_num_name_seoul = pd.read_csv('train_station_num_name_seoul.csv')

# 컬럼명 변경
station_num_name_seoul.columns=["역번호", "역사명", "노선명", "외부코드"]

station_num_name_seoul

,역번호,역사명,노선명,외부코드
0,244,용답,02호선,211-1
1,245,신답,02호선,211-2
2,250,용두,02호선,211-3
3,336,학여울,03호선,346
4,428,삼각지,04호선,428
...,...,...,...,...
762,159,동묘앞,01호선,127
763,200,까치산,02호선,234-4
764,201,시청,02호선,201
765,202,을지로입구,02호선,202


In [172]:
# 결측값 존재 확인
station_num_name_seoul.info()

# 확인 결과, 외부코드를 제외한 모든 값이 잘 들어있음 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   역번호     767 non-null    object
 1   역사명     767 non-null    object
 2   노선명     767 non-null    object
 3   외부코드    756 non-null    object
dtypes: object(4)
memory usage: 24.1+ KB


In [173]:
# 해당 데이터에 포함된 노선명 확인 및 둘 중 한 데이터셋에만 있는 노선명 확인
route_name_result = set(station_result['노선명'])
route_name_seoul = set(station_num_name_seoul["노선명"])
sub_route_name_result = route_name_result - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name_result

print(route_name_result)
print(route_name_seoul)

print(sub_route_name_result)
print(sub_route_name_seoul)

# 확인 결과, 공항철도, 경춘선 이름을 맞출 필요 있음
# 수인분당선, 경의선의 경우 다른, 다른 모든 API 확인한 뒤 통합할지 쪼갤지 고민 필요

{'신림선', '3호선', '인천2호선', '7호선', '8호선', '의정부선', '인천1호선', '경춘선', '1호선', '9호선', '5호선', '경의중앙선', '서해선', '에버라인선', '경강선', '2호선', '김포골드라인', '우이신설선', '신분당선', '4호선', '6호선', '수인분당선', '공항철도선'}
{'신림선', '공항철도', '06호선', '인천2호선', '07호선', '의정부경전철', '우이신설경전철', '용인경전철', '경춘선', '08호선', '01호선', '김포도시철도', '서해선', '04호선', '경의선', '경강선', '09호선', '03호선', '신분당선', '수인분당선', '02호선', '05호선', '인천선'}
{'3호선', '의정부선', '인천1호선', '9호선', '2호선', '김포골드라인', '우이신설선', '7호선', '5호선', '4호선', '경의중앙선', '6호선', '1호선', '공항철도선', '에버라인선', '8호선'}
{'01호선', '의정부경전철', '공항철도', '06호선', '09호선', '03호선', '용인경전철', '08호선', '김포도시철도', '인천선', '04호선', '02호선', '우이신설경전철', '07호선', '경의선', '05호선'}


In [174]:
# 일단 맞출 수 있는 부분은 통일
station_num_name_seoul["노선명"] = station_num_name_seoul["노선명"].replace({
    "우이신설경전철": "우이신설선",
    "의정부경전철": "의정부선",
    "용인경전철": "에버라인선",
    "김포도시철도": "김포골드라인",
    "인천선": "인천1호선",
    "공항철도": "공항철도선"
})
station_num_name_seoul["노선명"] = station_num_name_seoul["노선명"].str.replace(
    pat=r'^0', repl=r'', regex=True)

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name_result = set(station_result['노선명'])
route_name_seoul = set(station_num_name_seoul["노선명"])
sub_route_name_result = route_name_result - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name_result

print(route_name_result)
print(route_name_seoul)

print(sub_route_name_result)
print(sub_route_name_seoul)

{'신림선', '3호선', '인천2호선', '7호선', '8호선', '의정부선', '인천1호선', '경춘선', '1호선', '9호선', '5호선', '경의중앙선', '서해선', '에버라인선', '경강선', '2호선', '김포골드라인', '우이신설선', '신분당선', '4호선', '6호선', '수인분당선', '공항철도선'}
{'신림선', '3호선', '인천2호선', '7호선', '8호선', '의정부선', '인천1호선', '경춘선', '1호선', '9호선', '5호선', '서해선', '경의선', '에버라인선', '경강선', '2호선', '4호선', '신분당선', '김포골드라인', '우이신설선', '수인분당선', '6호선', '공항철도선'}
{'경의중앙선'}
{'경의선'}


In [175]:
# 역사명 맞추기
station_name_result = set(station_result['역사명'])
station_name_seoul = set(station_num_name_seoul['역사명'])
sub_station_name_result = station_name_result - station_name_seoul
sub_station_name_seoul = station_name_seoul - station_name_result

print(station_name_result)
print(station_name_seoul)

print(sub_station_name_result)
print(sub_station_name_seoul)

{'인천시청', '송정', '평택', '백마', '청라국제도시', '화정', '흥선', '덕소', '부평시장', '방학', '상도', '4.19민주묘지', '미금', '금천구청', '서울대벤처타운', '아신', '고색', '지제', '부천시청', '마곡나루', '천호(풍납토성)', '청량리(서울시립대입구)', '인천대공원', '운연(서창)', '동두천중앙', '세마', '문학경기장', '신당', '충정로(경기대입구)', '광명사거리', '청명', '부개', '송탄', '발산', '신대방', '당곡', '소요산', '신포', '신반포', '미아사거리', '사릉', '마들', '명지대', '개롱', '수리산', '의왕', '역촌', '왕십리(성동구청)', '역곡', '둔촌오륜', '김포공항', '상일동', '용두(동대문구청)', '대청', '아현', '보평', '신금호', '강동', '화전', '송파', '호구포', '샛강', '잠원', '죽전', '검바위', '장기', '소래포구', '성신여대입구(돈암)', '서부여성회관', '걸포북변', '이매', '문산', '시흥시청', '선부', '판교', '망원', '숭실대입구(살피재)', '신원', '면목', '상천', '가정중앙시장', '오목교(목동운동장앞)', '송내', '인천', '삼산체육관', '서동탄', '개봉', '기흥(백남준아트센터)', '가양', '경전철의정부', '신방화', '서현', '서빙고', '건대입구', '북한산보국문', '도화', '대곡', '원당', '왕길', '효자', '수원시청', '대흥(서강대앞)', '인천논현', '수서', '세류', '중랑', '광나루(장신대)', '금호', '을지로입구', '신창(순천향대)', '운천', '광운대', '경기광주', '백운', '상수', '아산', '매교', '압구정', '영통', '내방', '오산', '가정(루원시티)', '선정릉', '테크노파크', '고속터미널', '지평', '숭의', '초월', '한대앞', '곤지암', '경복궁(정부서울청사)', '

In [176]:
# 수동으로 이름 맞추기
station_num_name_seoul["역사명"] = station_num_name_seoul["역사명"].replace({
    "이촌": "이촌(국립중앙박물관)", 
    "4?19민주묘지": "4.19민주묘지",
    "청량리": "청량리(서울시립대입구)",
    "양재": "양재(서초구청)",
    "경복궁": "경복궁(정부서울청사)",
    "동대문역사문화공원": "동대문역사문화공원(DDP)",
    "구의": "구의(광진구청)",
    "굽은다리": "굽은다리(강동구민회관앞)",
    "월드컵경기장": "월드컵경기장(성산)",
    "화랑대": "화랑대(서울여대입구)",
    "군자": "군자(능동)",
    "총신대입구": "총신대입구(이수)",
    "이수": "총신대입구(이수)",
    "시민공원": "시민공원(문화창작지대)",
    "운연": "운연(서창)",
    "남부터미널": "남부터미널(예술의전당)",
    "광교": "광교(경기대)",
    "회현": "회현(남대문시장)",
    "하남시청": "하남시청(덕풍.신장)",
    "새절": "새절(신사)",
    "증산": "증산(명지대앞)",
    "상월곡": "상월곡(한국과학기술연구원)",
    "녹사평": "녹사평(용산구청)",
    "석남": "석남(거북시장)",
    "수유": "수유(강북구청)",
    "양재시민의숲": "양재시민의숲(매헌)",
    "교대": "교대(법원.검찰청)",
    "삼성": "삼성(무역센터)",
    "아시아드경기장": "아시아드경기장(공촌사거리)",
    "봉화산": "봉화산(서울의료원)",
    "성신여대입구": "성신여대입구(돈암)",
    "종로3가": "종로3가(탑골공원)",
    "동작": "동작(현충원)",
    "오목교": "오목교(목동운동장앞)",
    "기흥": "기흥(백남준아트센터)",
    "남한산성입구": "남한산성입구(성남법원.검찰청)",
    "용마산": "용마산(용마폭포공원)",
    "광나루": "광나루(장신대)",
    "낙성대": "낙성대(강감찬)",
    "숭실대입구": "숭실대입구(살피재)",
    "숙대입구": "숙대입구(갈월)",
    "충정로": "충정로(경기대입구)",
    "가정": "가정(루원시티)",
    "검단오류": "검단오류(검단산업단지)",
    "대흥": "대흥(서강대앞)",
    "온수": "온수(성공회대입구)",
    "왕십리": "왕십리(성동구청)",
    "천호": "천호(풍납토성)",
    "잠실": "잠실(송파구청)",
    "강변": "강변(동서울터미널)",
    "광흥창": "광흥창(서강)",
    "고려대": "고려대(종암)",
    "흑석": "흑석(중앙대입구)",
    "관악산": "관악산(서울대)",
    "미아": "미아(서울사이버대학)",
    "공릉": "공릉(서울과학기술대)",
    "사우": "사우(김포시청)",
    "상봉": "상봉(시외버스터미널)",
    "용두": "용두(동대문구청)",
    "운동장.송담대": "운동장.송담대(중앙시장)",
    "광교중앙": "광교중앙(아주대)",
    "동백": "동백(용인세브란스)",
    "아차산": "아차산(어린이대공원후문)",
    "신정": "신정(은행정)",
    "대림": "대림(구로구청)",
    "월곡": "월곡(동덕여대)",
    "한성대입구": "한성대입구(삼선교)",
    "올림픽공원": "올림픽공원(한국체대)",
    "신창": "신창(순천향대)",
    "쌍용": "쌍용(나사렛대)",
    "광화문": "광화문(세종문화회관)",
    "안암": "안암(고대병원앞)",
    "몽촌토성": "몽촌토성(평화의문)",
    "서울대입구": "서울대입구(관악구청)",
    "어린이대공원": "어린이대공원(세종대)"
})

# 역사명 재확인
# station_name_result = set(station_result['역사명'])
station_name_seoul = set(station_num_name_seoul['역사명'])
sub_station_name_result = station_name_result - station_name_seoul
sub_station_name_seoul = station_name_seoul - station_name_result

print(station_name_result)
print(station_name_seoul)

print(sub_station_name_result)
print(sub_station_name_seoul)

{'인천시청', '송정', '평택', '백마', '청라국제도시', '화정', '흥선', '덕소', '부평시장', '방학', '상도', '4.19민주묘지', '미금', '금천구청', '서울대벤처타운', '아신', '고색', '지제', '부천시청', '마곡나루', '천호(풍납토성)', '청량리(서울시립대입구)', '인천대공원', '운연(서창)', '동두천중앙', '세마', '문학경기장', '신당', '충정로(경기대입구)', '광명사거리', '청명', '부개', '송탄', '발산', '신대방', '당곡', '소요산', '신포', '신반포', '미아사거리', '사릉', '마들', '명지대', '개롱', '수리산', '의왕', '역촌', '왕십리(성동구청)', '역곡', '둔촌오륜', '김포공항', '상일동', '용두(동대문구청)', '대청', '아현', '보평', '신금호', '강동', '화전', '송파', '호구포', '샛강', '잠원', '죽전', '검바위', '장기', '소래포구', '성신여대입구(돈암)', '서부여성회관', '걸포북변', '이매', '문산', '시흥시청', '선부', '판교', '망원', '숭실대입구(살피재)', '신원', '면목', '상천', '가정중앙시장', '오목교(목동운동장앞)', '송내', '인천', '삼산체육관', '서동탄', '개봉', '기흥(백남준아트센터)', '가양', '경전철의정부', '신방화', '서현', '서빙고', '건대입구', '북한산보국문', '도화', '대곡', '원당', '왕길', '효자', '수원시청', '대흥(서강대앞)', '인천논현', '수서', '세류', '중랑', '광나루(장신대)', '금호', '을지로입구', '신창(순천향대)', '운천', '광운대', '경기광주', '백운', '상수', '아산', '매교', '압구정', '영통', '내방', '오산', '가정(루원시티)', '선정릉', '테크노파크', '고속터미널', '지평', '숭의', '초월', '한대앞', '곤지암', '경복궁(정부서울청사)', '

In [177]:
station_result[(station_result["역사명"] == "운천") |
               (station_result["역사명"] == "지제") | (station_result["역사명"] == "평택지제")]

# 지제 -> 평택지제 이름 변경
station_result["역사명"] = station_result["역사명"].replace({
    "지제": "평택지제"
})

station_result[(station_result["역사명"] == "운천") |
               (station_result["역사명"] == "지제") | (station_result["역사명"] == "평택지제")]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도
496,1723,평택지제,I4101,1호선,일반역,NaN,NaN,경기도 평택시 경기대로 777,1723.0,37.018800,127.070444
557,1286,운천,I4108,경의중앙선,일반역,NaN,NaN,경기도 파주시 문산읍 통일로2033번길3,1286.0,37.879942,126.769999


In [178]:
station_num_name_seoul[(station_num_name_seoul["역사명"] == "운천") |
                       (station_num_name_seoul["역사명"] == "지제") | (station_num_name_seoul["역사명"] == "평택지제")]

# 수동으로 운천 역번호 추가
station_num_name_seoul.loc[767] = ['1286', '운천', '경의선', np.NaN]

station_num_name_seoul[(station_num_name_seoul["역사명"] == "운천") |
                       (station_num_name_seoul["역사명"] == "지제") | (station_num_name_seoul["역사명"] == "평택지제")]

,역번호,역사명,노선명,외부코드
347,1723,평택지제,1호선,P164
767,1286,운천,경의선,NaN


In [179]:
# 두 값 합치기
station_temp = pd.merge(station_result, station_num_name_seoul, on=['역사명', '노선명'], how='left')
station_temp = station_temp.astype({'역번호_y': 'str'})

station_temp = station_temp.reset_index(drop=True)
station_temp

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도,역번호,외부코드
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,인천광역시 계양구 다남로 24,3110.0,37.571449,126.735780,3110,I110
1,3111,귤현,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 1136,3111.0,37.566379,126.742654,3111,I111
2,3112,박촌,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 992,3112.0,37.553703,126.745077,3112,I112
3,3113,임학,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 875,3113.0,37.545059,126.738665,3113,I113
4,3114,계산,S2801,인천1호선,일반역,-,-,인천광역시 계양구 경명대로 1089,3114.0,37.543238,126.728128,3114,I114
...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,1008,이촌(국립중앙박물관),I4108,경의중앙선,환승역,L2604,4호선,서울시 용산구 이촌로 71길 42,1008.0,37.522427,126.973406,NaN,NaN
762,1009,서빙고,I4108,경의중앙선,일반역,NaN,NaN,서울시 용산구 서빙고로 238(서빙고동 241-11),1009.0,37.519594,126.988537,NaN,NaN
763,1010,한남,I4108,경의중앙선,일반역,NaN,NaN,서울시 용산구 독서당로6길 12-13(한남동),1010.0,37.529430,127.009169,NaN,NaN
764,1011,옥수,I4108,경의중앙선,환승역,S2603,3호선,서울시 성동구 동호로 지하21(옥수동),1011.0,37.540446,127.018672,NaN,NaN


In [180]:
# 역번호_y 데이터형식 맞추기
station_temp = station_temp.astype({'역번호_y': 'str'})
station_temp['역번호_y'] = station_temp['역번호_y'].str.replace(
    pat=r'.0$', repl=r'', regex=True)

station_temp 

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도,역번호,외부코드
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,인천광역시 계양구 다남로 24,3110,37.571449,126.735780,3110,I110
1,3111,귤현,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 1136,3111,37.566379,126.742654,3111,I111
2,3112,박촌,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 992,3112,37.553703,126.745077,3112,I112
3,3113,임학,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 875,3113,37.545059,126.738665,3113,I113
4,3114,계산,S2801,인천1호선,일반역,-,-,인천광역시 계양구 경명대로 1089,3114,37.543238,126.728128,3114,I114
...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,1008,이촌(국립중앙박물관),I4108,경의중앙선,환승역,L2604,4호선,서울시 용산구 이촌로 71길 42,1008,37.522427,126.973406,NaN,NaN
762,1009,서빙고,I4108,경의중앙선,일반역,NaN,NaN,서울시 용산구 서빙고로 238(서빙고동 241-11),1009,37.519594,126.988537,NaN,NaN
763,1010,한남,I4108,경의중앙선,일반역,NaN,NaN,서울시 용산구 독서당로6길 12-13(한남동),1010,37.529430,127.009169,NaN,NaN
764,1011,옥수,I4108,경의중앙선,환승역,S2603,3호선,서울시 성동구 동호로 지하21(옥수동),1011,37.540446,127.018672,NaN,NaN


In [181]:
# 역번호_y != 역번호인 노선 몇 개 있는지 각각 확인해보기
null_station_cnt_by_route = station_temp['노선명'][station_temp['역번호'] != station_temp['역번호_y']].value_counts().sort_index()
null_station_cnt_by_route

1호선       3
5호선       2
경의중앙선    57
경춘선       5
서해선       2
수인분당선    10
Name: 노선명, dtype: int64

In [182]:
# 1호선
station_temp[(station_temp['노선명'] == '1호선') & (station_temp['역번호'] != station_temp['역번호_y'])]

# 확인 결과, 역번호가 맞음

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도,역번호,외부코드
521,1015,회기,I4101,1호선,환승역,"I4108, I41K2","경의중앙선, 경춘선",서울시 동대문구 회기로196(휘경동 317-101),nan,37.589802,127.057936,1015,123
575,1821,온수(성공회대입구),I1101,1호선,환승역,S1107,7호선,서울시 구로구 부일로 872,nan,37.492267,126.823399,1821,145
576,1803,역곡,I1101,1호선,일반역,NaN,NaN,경기도 부천시 원미구 역곡로 1(역곡동),1821,37.492433,126.824086,1803,146


In [183]:
# 5호선
station_temp[(station_temp['노선명'] == '5호선') & (
    station_temp['역번호'] != station_temp['역번호_y'])]

# 확인 결과, 역번호가 맞음 

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도,역번호,외부코드
323,0557,하남시청(덕풍.신장),S1105,5호선,도시철도 일반역,NaN,NaN,경기도 하남시 하남대로 지하820(덕풍동),2566,37.54205,127.20612,2565,557
324,0558,하남검단산,S1105,5호선,도시철도 일반역,NaN,NaN,경기도 하남시 대청로 지하100(창우동),2565,37.53972,127.22345,2566,558


In [184]:
# 경의중앙
station_temp[(station_temp['노선명'] == '경의중앙선') & (station_temp['역번호'] != station_temp['역번호_y'])]

# 경중선은 아차피 역번호로 시간표 찾기 제공하지 않음

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도,역번호,외부코드
545,1285,임진강,I4108,경의중앙선,일반역,NaN,NaN,경기도 파주시 문산읍 임진각로 115(문산읍),1285,37.888421,126.746765,NaN,NaN
546,1286,운천,I4108,경의중앙선,일반역,NaN,NaN,경기도 파주시 문산읍 통일로2033번길3,1286,37.879942,126.769999,NaN,NaN
547,1251,서울역,I4108,경의중앙선,환승역,"S2601, L2604","1호선, 4호선",서울시 용산구 한강대로 405,1251,37.557231,126.971030,NaN,NaN
548,1252,신촌,I4108,경의중앙선,일반역,NaN,NaN,서울특별시 서대문구 대현동 74-12,1252,37.559733,126.942597,NaN,NaN
549,1265,가좌,I4108,경의중앙선,환승역,NaN,NaN,서울시 서대문구 수색로 27(남가좌동),1265,37.568491,126.915487,NaN,NaN
550,1261,효창공원앞,I4108,경의중앙선,환승역,S1106,6호선,서울시 용산구 용문동 5-133,1261,37.538579,126.962210,NaN,NaN
551,1262,공덕,I4108,경의중앙선,환승역,"S1105, S1106, I28A1","5호선, 6호선, 공항",서울시 마포구 공덕동 마포대로 자하100,1262,37.542596,126.952099,NaN,NaN
552,1263,서강대,I4108,경의중앙선,일반역,NaN,NaN,서울시 마포구 서강로 지하118,1263,37.551881,126.935711,NaN,NaN
553,239,홍대입구,I4108,경의중앙선,환승역,"S2602, I28A1","2호선, 공항",서울시 마포구 양화로 지하 188,1264,37.557641,126.926683,NaN,NaN
554,1266,디지털미디어시티,I4108,경의중앙선,환승역,"S1106, I28A1","6호선, 공항",서울시 은평구 수색로 193(수색동 37-2),1266,37.577475,126.900453,NaN,NaN


In [185]:
# 경춘선
station_temp[(station_temp['노선명'] == '경춘선') & (station_temp['역번호'] != station_temp['역번호_y'])]

# 경춘선은 아차피 역번호로 시간표 찾기 제공하지 않음

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도,역번호,외부코드
592,1015,회기,I41K2,경춘선,환승역,"I4108, I4101","경의중앙선, 경원선",서울시 동대문구 회기로196(휘경동 317-101),nan,37.589802,127.057936,1307,P118
593,1014,청량리(서울시립대입구),I41K2,경춘선,환승역,"S2601, I4108, I28K1","1호선, 경의중앙선, 분당선",서울시 동대문구 왕산로 214,nan,37.580543,127.046516,1306,P117
594,1201,중랑,I41K2,경춘선,환승역,I4108,경의중앙선,서울특별시 중랑구 중화동 321-37,nan,37.594948,127.076152,1308,P119
595,1202,상봉(시외버스터미널),I41K2,경춘선,환승역,"S1107, I4108","7호선, 경의중앙선",서울시 중랑구 망우로 297(상봉1동 100-9번지),nan,37.595254,127.085853,1309,P120
596,1203,망우,I41K2,경춘선,환승역,I4108,경의중앙선,서울시 중랑구 망우로 55길 11-10,nan,37.599288,127.092318,1310,P121


In [186]:
# 서해선
station_temp[(station_temp['노선명'] == '서해선') & (station_temp['역번호'] != station_temp['역번호_y'])]

# 서해선은 어차피 역번호로 시간표 제공하지 않음

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도,역번호,외부코드
720,4814,시우,I41WS,서해선,일반역,NaN,NaN,경기도 안산시 단원구 동산로 지하 50,nan,37.313767,126.798303,4814,701
721,4815,원시,I41WS,서해선,일반역,NaN,NaN,경기도 안산시 단원구 선단로 지하 70,4814,37.313210,126.796261,4815,702


In [187]:
# 수인분당선
station_temp[(station_temp['노선명'] == '수인분당선') & (
    station_temp['역번호'] != station_temp['역번호_y'])]

# 수인분당선도 어차피 제공하지 않음

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,역번호_y,위도,경도,역번호,외부코드
616,1014,청량리(서울시립대입구),I28K1,수인분당선,환승역,"S2601, I41K2, I4108","1호선, 경춘선, 경의중앙선",서울시 동대문구 왕산로 214,nan,37.580543,127.046516,1845,K209
617,1013,왕십리(성동구청),I28K1,수인분당선,환승역,"S2602, S1105, I4108","2호선, 5호선, 경의중앙선",서울시 성동구 왕십리 광장로 17(행당동),nan,37.561128,127.035505,102C,K210
658,1755,한대앞,I28K1,수인분당선,일반역,NaN,NaN,경기도 안산시 상록구 충장로 337,nan,37.309685,126.853656,1830,K251
659,1756,중앙,I28K1,수인분당선,일반역,NaN,NaN,경기도 안산시 단원구 중앙대로 918,nan,37.316055,126.838532,1831,K252
660,1757,고잔,I28K1,수인분당선,일반역,NaN,NaN,경기도 안산시 상록구 충장로 337,nan,37.316821,126.823094,1832,K253
661,1758,초지,I28K1,수인분당선,환승역,"I4103, I41WS","안산과천선, 서해선",경기도 안산시 단원구 중앙대로 620,nan,37.320583,126.806151,1833,K254
662,1759,안산,I28K1,수인분당선,일반역,NaN,NaN,경기도 안산시 단원구 중앙대로 620,nan,37.327006,126.788748,1834,K255
663,1760,신길온천,I28K1,수인분당선,일반역,NaN,NaN,경기도 안산시 단원구 중앙대로 462,nan,37.337488,126.767306,1835,K256
664,1761,정왕,I28K1,수인분당선,일반역,NaN,NaN,경기도 안산시 단원구 황고개로 2,nan,37.351816,126.742844,1836,K257
665,1762,오이도,I28K1,수인분당선,일반역,NaN,NaN,경기도 시흥시 역전로 430,nan,37.361773,126.738437,1800,K258


In [188]:
# 역번호_y == 역번호 == NaN인 역 확인 <- X
station_temp[(station_temp['역번호'].isnull()) & (station_temp['역번호_y'].isnull())]

# 역번호 == NaN인 역 확인 <- 경의중앙선 
station_temp[(station_temp['역번호'].isnull())]

# 즉, 그냥 역번호_y 대신 역번호 사용하면 됨 
station_result = station_temp
station_result.drop(columns=["역번호_x", "역번호_y"], inplace=True)

station_result

,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드
0,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,인천광역시 계양구 다남로 24,37.571449,126.735780,3110,I110
1,귤현,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 1136,37.566379,126.742654,3111,I111
2,박촌,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 992,37.553703,126.745077,3112,I112
3,임학,S2801,인천1호선,일반역,-,-,인천광역시 계양구 장제로 875,37.545059,126.738665,3113,I113
4,계산,S2801,인천1호선,일반역,-,-,인천광역시 계양구 경명대로 1089,37.543238,126.728128,3114,I114
...,...,...,...,...,...,...,...,...,...,...,...
761,이촌(국립중앙박물관),I4108,경의중앙선,환승역,L2604,4호선,서울시 용산구 이촌로 71길 42,37.522427,126.973406,NaN,NaN
762,서빙고,I4108,경의중앙선,일반역,NaN,NaN,서울시 용산구 서빙고로 238(서빙고동 241-11),37.519594,126.988537,NaN,NaN
763,한남,I4108,경의중앙선,일반역,NaN,NaN,서울시 용산구 독서당로6길 12-13(한남동),37.529430,127.009169,NaN,NaN
764,옥수,I4108,경의중앙선,환승역,S2603,3호선,서울시 성동구 동호로 지하21(옥수동),37.540446,127.018672,NaN,NaN


In [189]:
# 역번호 수정
station_result["역번호"][station_result["역번호"].str.len() < 4] = "0" + station_result["역번호"][station_result["역번호"].str.len() < 4]

# 확인 결과, 제대로 해결
station_result[station_result["역번호"].str.len() < 4]

C:\Users\04746\AppData\Local\Temp\ipykernel_15580\3857605931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_result["역번호"][station_result["역번호"].str.len() < 4] = "0" + station_result["역번호"][station_result["역번호"].str.len() < 4]


,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드


### 6. 실제 노선도와 비교

In [190]:
# 역이 노선별로 몇 개인지 확인
total_station_cnt_by_route = station_result["노선명"].value_counts(
).sort_index()
total_station_cnt_by_route

1호선       99
2호선       50
3호선       44
4호선       51
5호선       56
6호선       39
7호선       53
8호선       18
9호선       38
경강선       11
경의중앙선     57
경춘선       24
공항철도선     14
김포골드라인    10
서해선       12
수인분당선     63
신림선       11
신분당선      16
에버라인선     15
우이신설선     13
의정부선      15
인천1호선     30
인천2호선     27
Name: 노선명, dtype: int64

In [191]:
# 1호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "1호선"]))

['청량리(서울시립대입구)', '제기동', '신설동', '동묘앞', '동대문', '종로5가', '종로3가(탑골공원)', '종각', '시청', '서울역', '수원', '세류', '병점', '서동탄', '세마', '오산대', '오산', '진위', '송탄', '평택지제', '서정리', '평택', '명학', '성환', '직산', '두정', '천안', '봉명', '쌍용(나사렛대)', '아산', '탕정', '배방', '온양온천', '신창(순천향대)', '광명', '석수', '관악', '안양', '남영', '용산', '노량진', '대방', '신길', '영등포', '신도림', '구로', '가산디지털단지', '독산', '금천구청', '금정', '군포', '당정', '의왕', '성균관대', '화서', '회기', '외대앞', '신이문', '석계', '광운대', '월계', '녹천', '창동', '방학', '도봉', '도봉산', '망월사', '회룡', '의정부', '가능', '녹양', '양주', '덕계', '덕정', '지행', '동두천중앙', '보산', '동두천', '소요산', '구일', '개봉', '오류동', '온수(성공회대입구)', '역곡', '소사', '부천', '중동', '송내', '부개', '부평', '백운', '동암', '간석', '주안', '도화', '제물포', '도원', '동인천', '인천']


In [192]:
# 2호선
print(list(station_result["역사명"][station_result["노선명"] == "2호선"]))

# 2호선 까치산 없음, 추가 필요
station_result[station_result["역사명"] == "까치산"]

['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원(DDP)', '신당', '상왕십리', '왕십리(성동구청)', '한양대', '뚝섬', '성수', '건대입구', '구의(광진구청)', '강변(동서울터미널)', '잠실나루', '잠실(송파구청)', '잠실새내', '종합운동장', '삼성(무역센터)', '선릉', '역삼', '강남', '교대(법원.검찰청)', '서초', '방배', '사당', '낙성대(강감찬)', '서울대입구(관악구청)', '봉천', '신림', '신대방', '구로디지털단지', '대림(구로구청)', '신도림', '문래', '영등포구청', '당산', '합정', '홍대입구', '신촌', '이대', '아현', '충정로(경기대입구)', '용답', '신답', '용두(동대문구청)', '신설동', '도림천', '양천구청', '신정네거리']


,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드
278,까치산,S1105,5호선,도시철도 환승역,S1102,수도권 도시철도 2호선,서울특별시 강서구 강서로 지하54(화곡동),37.531768,126.846683,2519,518


In [193]:
# 역번호 찾아오기 = 0200
station_num_name_seoul[station_num_name_seoul["역사명"] == "까치산"]

# 정보 수동 입력해서 추가해주기
station_result.loc[766] = ["까치산", np.NaN, "2호선", np.NaN,
                           np.NaN, np.NaN, "서울특별시 강서구 강서로 54", 37.531394, 126.846987, "0200", np.NaN]

# 재확인
station_result[station_result["역사명"] == "까치산"]

,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드
278,까치산,S1105,5호선,도시철도 환승역,S1102,수도권 도시철도 2호선,서울특별시 강서구 강서로 지하54(화곡동),37.531768,126.846683,2519,518
766,까치산,NaN,2호선,NaN,NaN,NaN,서울특별시 강서구 강서로 54,37.531394,126.846987,0200,NaN


In [194]:
# 3호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "3호선"]))

['지축', '구파발', '연신내', '불광', '녹번', '홍제', '무악재', '독립문', '경복궁(정부서울청사)', '안국', '종로3가(탑골공원)', '을지로3가', '충무로', '동대입구', '약수', '금호', '옥수', '압구정', '신사', '잠원', '고속터미널', '교대(법원.검찰청)', '남부터미널(예술의전당)', '양재(서초구청)', '매봉', '도곡', '대치', '학여울', '대청', '일원', '수서', '가락시장', '경찰병원', '오금', '삼송', '원흥', '원당', '화정', '대곡', '백석', '마두', '정발산', '주엽', '대화']


In [195]:
# 4호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "4호선"]))

['별내별가람', '오남', '진접', '당고개', '상계', '노원', '창동', '쌍문', '수유(강북구청)', '미아(서울사이버대학)', '미아사거리', '길음', '성신여대입구(돈암)', '한성대입구(삼선교)', '혜화', '동대문', '동대문역사문화공원(DDP)', '충무로', '명동', '회현(남대문시장)', '서울역', '숙대입구(갈월)', '삼각지', '신용산', '이촌(국립중앙박물관)', '동작(현충원)', '총신대입구(이수)', '사당', '남태령', '선바위', '경마공원', '대공원', '과천', '정부과천청사', '인덕원', '평촌', '범계', '산본', '수리산', '대야미', '반월', '상록수', '중앙', '한대앞', '고잔', '초지', '안산', '신길온천', '정왕', '오이도', '금정']


In [196]:
# 5호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "5호선"]))

['방화', '개화산', '김포공항', '송정', '마곡', '발산', '우장산', '화곡', '까치산', '신정(은행정)', '목동', '오목교(목동운동장앞)', '양평', '영등포구청', '영등포시장', '신길', '여의도', '여의나루', '마포', '공덕', '애오개', '충정로(경기대입구)', '서대문', '광화문(세종문화회관)', '종로3가(탑골공원)', '을지로4가', '동대문역사문화공원(DDP)', '청구', '신금호', '행당', '왕십리(성동구청)', '마장', '답십리', '장한평', '군자(능동)', '아차산(어린이대공원후문)', '광나루(장신대)', '천호(풍납토성)', '강동', '길동', '굽은다리(강동구민회관앞)', '명일', '고덕', '상일동', '둔촌동', '올림픽공원(한국체대)', '방이', '오금', '개롱', '거여', '강일', '미사', '하남풍산', '하남시청(덕풍.신장)', '하남검단산', '마천']


In [197]:
# 6호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "6호선"]))

['응암', '역촌', '불광', '독바위', '연신내', '구산', '새절(신사)', '증산(명지대앞)', '디지털미디어시티', '월드컵경기장(성산)', '마포구청', '망원', '합정', '상수', '광흥창(서강)', '대흥(서강대앞)', '공덕', '효창공원앞', '삼각지', '녹사평(용산구청)', '이태원', '한강진', '버티고개', '약수', '청구', '신당', '동묘앞', '창신', '보문', '안암(고대병원앞)', '고려대(종암)', '월곡(동덕여대)', '상월곡(한국과학기술연구원)', '돌곶이', '석계', '태릉입구', '화랑대(서울여대입구)', '봉화산(서울의료원)', '신내']


In [198]:
# 7호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "7호선"]))

['까치울', '부천종합운동장', '춘의', '신중동', '부천시청', '상동', '삼산체육관', '굴포천', '부평구청', '산곡', '석남(거북시장)', '장암', '도봉산', '수락산', '마들', '노원', '중계', '하계', '공릉(서울과학기술대)', '태릉입구', '먹골', '중화', '상봉(시외버스터미널)', '면목', '사가정', '용마산(용마폭포공원)', '중곡', '군자(능동)', '어린이대공원(세종대)', '건대입구', '뚝섬유원지', '청담', '강남구청', '학동', '논현', '반포', '고속터미널', '내방', '총신대입구(이수)', '남성', '숭실대입구(살피재)', '상도', '장승배기', '신대방삼거리', '보라매', '신풍', '대림(구로구청)', '남구로', '가산디지털단지', '철산', '광명사거리', '천왕', '온수(성공회대입구)']


In [199]:
# 8호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "8호선"]))

['암사', '천호(풍납토성)', '강동구청', '몽촌토성(평화의문)', '잠실(송파구청)', '석촌', '송파', '가락시장', '문정', '장지', '복정', '남위례', '산성', '남한산성입구(성남법원.검찰청)', '단대오거리', '신흥', '수진', '모란']


In [200]:
# 9호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "9호선"]))

['개화', '김포공항', '공항시장', '신방화', '마곡나루', '양천향교', '가양', '증미', '등촌', '염창', '신목동', '선유도', '당산', '국회의사당', '여의도', '샛강', '노량진', '노들', '흑석(중앙대입구)', '동작(현충원)', '구반포', '신반포', '고속터미널', '사평', '신논현', '중앙보훈병원', '둔촌오륜', '올림픽공원(한국체대)', '한성백제', '송파나루', '석촌', '석촌고분', '삼전', '종합운동장', '봉은사', '삼성중앙', '선정릉', '언주']


In [201]:
# 경강선 - 모두 존재 
print(list(station_result["역사명"][station_result["노선명"] == "경강선"]))

['판교', '이매', '삼동', '경기광주', '초월', '곤지암', '신둔도예촌', '이천', '부발', '세종대왕릉', '여주']


In [202]:
# 경의중앙선 
print(list(station_result["역사명"][station_result["노선명"] == "경의중앙선"]))

# 도라산 빠짐, 추가 필요
station_result[station_result["역사명"] == "도라산"]

['임진강', '운천', '서울역', '신촌', '가좌', '효창공원앞', '공덕', '서강대', '홍대입구', '디지털미디어시티', '수색', '화전', '강매', '행신', '능곡', '곡산', '백마', '풍산', '일산', '탄현', '야당', '운정', '금릉', '금촌', '월롱', '파주', '문산', '용산', '왕십리(성동구청)', '청량리(서울시립대입구)', '회기', '중랑', '상봉(시외버스터미널)', '망우', '양원', '지평', '구리', '도농', '양정', '덕소', '도심', '팔당', '운길산', '양수', '신원', '국수', '아신', '오빈', '양평', '원덕', '용문', '대곡', '이촌(국립중앙박물관)', '서빙고', '한남', '옥수', '응봉']


,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드


In [203]:
# 역번호 없음 (어차피 필요 없음)
station_num_name_seoul[station_num_name_seoul["역사명"] == "도라산"]

# 정보 수동 입력해서 추가해주기
station_result.loc[767] = ["도라산", np.NaN, "경의중앙선", np.NaN,
                           np.NaN, np.NaN, "경기도 파주시 장단면 희망로 307", 37.898307, 126.709193, np.NaN, np.NaN]

# 재확인
station_result[station_result["역사명"] == "도라산"]

,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드
767,도라산,NaN,경의중앙선,NaN,NaN,NaN,경기도 파주시 장단면 희망로 307,37.898307,126.709193,NaN,NaN


In [204]:
# 경춘선 
print(list(station_result["역사명"][station_result["노선명"] == "경춘선"]))

# 광운대 빠짐, 추가 필요
station_result[station_result["역사명"] == "광운대"]

['회기', '청량리(서울시립대입구)', '중랑', '상봉(시외버스터미널)', '망우', '신내', '갈매', '별내', '퇴계원', '사릉', '금곡', '평내호평', '천마산', '마석', '대성리', '청평', '상천', '가평', '굴봉산', '백양리', '강촌', '김유정', '남춘천', '춘천']


,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드
525,광운대,I4101,1호선,일반역,NaN,NaN,서울시 노원구 석계로 98-2(광운대역),37.623632,127.061835,1019,119


In [205]:
# 역번호 1305
station_num_name_seoul[station_num_name_seoul["역사명"] == "광운대"]

# 정보 수동 입력해서 추가해주기
station_result.loc[768] = ["광운대", np.NaN, "경춘선", np.NaN,
                           np.NaN, np.NaN, "서울특별시 노원구 석계로 98-2", 37.623986, 127.061956, "1305", np.NaN]

# 재확인
station_result[station_result["역사명"] == "광운대"]

,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드
525,광운대,I4101,1호선,일반역,NaN,NaN,서울시 노원구 석계로 98-2(광운대역),37.623632,127.061835,1019,119
768,광운대,NaN,경춘선,NaN,NaN,NaN,서울특별시 노원구 석계로 98-2,37.623986,127.061956,1305,NaN


In [206]:
# 공항철도선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "공항철도선"]))

['서울역', '공덕', '홍대입구', '디지털미디어시티', '마곡나루', '김포공항', '계양', '검암', '청라국제도시', '영종', '운서', '공항화물청사', '인천공항1터미널', '인천공항2터미널']


In [207]:
# 김포골드라인 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "김포골드라인"]))

['양촌', '구래', '마산', '장기', '운양', '걸포북변', '사우(김포시청)', '풍무', '고촌', '김포공항']


In [208]:
# 서해선 - 모두 존재 
print(list(station_result["역사명"][station_result["노선명"] == "서해선"]))

['소사', '소새울', '시흥대야', '신천', '신현', '시흥시청', '시흥능곡', '달미', '선부', '초지', '시우', '원시']


In [209]:
# 수인분당선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "수인분당선"]))

['청량리(서울시립대입구)', '왕십리(성동구청)', '서울숲', '압구정로데오', '강남구청', '선정릉', '선릉', '한티', '도곡', '구룡', '개포동', '대모산입구', '수서', '복정', '가천대', '태평', '모란', '야탑', '이매', '서현', '수내', '정자', '미금', '오리', '죽전', '보정', '구성', '신갈', '기흥(백남준아트센터)', '상갈', '청명', '영통', '망포', '매탄권선', '수원시청', '매교', '수원', '고색', '오목천', '어천', '야목', '사리', '한대앞', '중앙', '고잔', '초지', '안산', '신길온천', '정왕', '오이도', '달월', '월곶', '소래포구', '인천논현', '호구포', '남동인더스파크', '원인재', '연수', '송도', '인하대', '숭의', '신포', '인천']


In [210]:
# 신림선 - 모두 존재 
print(list(station_result["역사명"][station_result["노선명"] == "신림선"]))

['샛강', '대방', '서울지방병무청', '보라매', '보라매공원', '보라매병원', '당곡', '신림', '서원', '서울대벤처타운', '관악산(서울대)']


In [211]:
# 신분당선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "신분당선"]))

['신사', '논현', '신논현', '강남', '양재(서초구청)', '양재시민의숲(매헌)', '청계산입구', '판교', '정자', '미금', '동천', '수지구청', '성복', '상현', '광교중앙(아주대)', '광교(경기대)']


In [212]:
# 에버라인선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "에버라인선"]))

['기흥(백남준아트센터)', '강남대', '지석', '어정', '동백(용인세브란스)', '초당', '삼가', '시청.용인대', '명지대', '김량장', '운동장.송담대(중앙시장)', '고진', '보평', '둔전', '전대.에버랜드']


In [213]:
# 우이신설선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "우이신설선"]))

['북한산우이', '솔밭공원', '4.19민주묘지', '가오리', '화계', '삼양', '삼양사거리', '솔샘', '북한산보국문', '정릉', '성신여대입구(돈암)', '보문', '신설동']


In [214]:
# 의정부선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "의정부선"]))

['발곡', '회룡', '범골', '경전철의정부', '의정부시청', '흥선', '의정부중앙', '동오', '새말', '경기도청북부청사', '효자', '곤제', '어룡', '송산', '탑석']


In [215]:
# 인천1호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "인천1호선"]))

['계양', '귤현', '박촌', '임학', '계산', '경인교대입구', '작전', '갈산', '부평구청', '부평시장', '부평', '동수', '부평삼거리', '간석오거리', '인천시청', '예술회관', '인천터미널', '문학경기장', '선학', '신연수', '원인재', '동춘', '동막', '캠퍼스타운', '테크노파크', '지식정보단지', '인천대입구', '센트럴파크', '국제업무지구', '송도달빛축제공원']


In [216]:
# 인천2호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "인천2호선"]))

['검단오류(검단산업단지)', '왕길', '검단사거리', '마전', '완정', '독정', '검암', '검바위', '아시아드경기장(공촌사거리)', '서구청', '가정(루원시티)', '가정중앙시장', '석남(거북시장)', '서부여성회관', '인천가좌', '가재울', '주안국가산단', '주안', '시민공원(문화창작지대)', '석바위시장', '인천시청', '석천사거리', '모래내시장', '만수', '남동구청', '인천대공원', '운연(서창)']


### 7. 서울 근교역 추리기

##### 7-a. 주소 결측값 확인

In [217]:
null_address_station_cnt_by_route = station_result['노선명'][station_result['주소'].isnull(
)].value_counts().sort_index()

null_address_station_cnt_by_route

4호선     1
9호선    13
Name: 노선명, dtype: int64

In [218]:
station_result[station_result['주소'].isnull()]

,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드
722,중앙보훈병원,NaN,9호선,NaN,NaN,NaN,NaN,37.529191,127.148739,4138,938
723,둔촌오륜,NaN,9호선,NaN,NaN,NaN,NaN,37.519683,127.137989,4137,937
724,올림픽공원(한국체대),NaN,9호선,NaN,NaN,NaN,NaN,37.516269,127.130288,4136,936
725,한성백제,NaN,9호선,NaN,NaN,NaN,NaN,37.516404,127.116503,4135,935
726,송파나루,NaN,9호선,NaN,NaN,NaN,NaN,37.510372,127.112216,4134,934
727,석촌,NaN,9호선,NaN,NaN,NaN,NaN,37.505208,127.107040,4133,933
728,석촌고분,NaN,9호선,NaN,NaN,NaN,NaN,37.502558,127.097033,4132,932
729,삼전,NaN,9호선,NaN,NaN,NaN,NaN,37.504738,127.088025,4131,931
730,종합운동장,NaN,9호선,NaN,NaN,NaN,NaN,37.511426,127.076275,4130,930
731,봉은사,NaN,9호선,NaN,NaN,NaN,NaN,37.514219,127.060245,4129,929


In [219]:
# 수동으로 추가
station_result.iloc[722, 6] = '서울특별시 강동구 둔촌동 8-1'
station_result.iloc[723, 6] = '서울특별시 강동구 둔촌동 227-7'
station_result.iloc[724, 6] = '서울특별시 송파구 방이동 89-28'
station_result.iloc[725, 6] = '서울특별시 송파구 방이동 88-17'
station_result.iloc[726, 6] = '서울특별시 송파구 방이동 2'
station_result.iloc[727, 6] = '서울특별시 송파구 석촌동 209'
station_result.iloc[728, 6] = '서울특별시 송파구 삼전동 157-1'
station_result.iloc[729, 6] = '서울특별시 송파구 잠실동 347'
station_result.iloc[730, 6] = '서울특별시 송파구 잠실동 123'
station_result.iloc[731, 6] = '서울특별시 강남구 삼성동 172'
station_result.iloc[732, 6] = '서울특별시 강남구 삼성동 111-147'
station_result.iloc[733, 6] = '서울특별시 강남구 삼성동 111-114'
station_result.iloc[734, 6] = '서울특별시 강남구 논현동 279-165'
station_result.iloc[735, 6] = '경기도 군포시 군포로 750'

# 확인 결과, 주소 결측값 없음
station_result[station_result['주소'].isnull()]

,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,주소,위도,경도,역번호,외부코드


### 8. 필요 없는 컬럼 제거 

In [220]:
station_result.drop(columns=["노선번호", "환승역구분", "환승노선번호", "환승노선명"], inplace=True)
station_result

,역사명,노선명,주소,위도,경도,역번호,외부코드
0,계양,인천1호선,인천광역시 계양구 다남로 24,37.571449,126.735780,3110,I110
1,귤현,인천1호선,인천광역시 계양구 장제로 1136,37.566379,126.742654,3111,I111
2,박촌,인천1호선,인천광역시 계양구 장제로 992,37.553703,126.745077,3112,I112
3,임학,인천1호선,인천광역시 계양구 장제로 875,37.545059,126.738665,3113,I113
4,계산,인천1호선,인천광역시 계양구 경명대로 1089,37.543238,126.728128,3114,I114
...,...,...,...,...,...,...,...
764,옥수,경의중앙선,서울시 성동구 동호로 지하21(옥수동),37.540446,127.018672,NaN,NaN
765,응봉,경의중앙선,서울시 성동구 고산자로 123(응봉동),37.549946,127.034538,NaN,NaN
766,까치산,2호선,서울특별시 강서구 강서로 54,37.531394,126.846987,0200,NaN
767,도라산,경의중앙선,경기도 파주시 장단면 희망로 307,37.898307,126.709193,NaN,NaN


### 9. 파일 생성

In [221]:
station_result.to_csv("result_train_station.csv")